In [1]:
import bql
import numpy as np
from IPython.display import display, clear_output
import pandas as pd
import os
import time
import bqport
from datetime import date, datetime, timedelta
import datetime
from ipywidgets import *
from  bqwidgets import AutoComplete, DatePicker
import ipywidgets
from openpyxl import load_workbook
import shutil

bq = bql.Service()
d = bq.data
f = bq.func
u = bq.univ

In [2]:
def bq_res_array_to_pd(res_array):
    a = res_array[0].df().reset_index()
    r_all = pd.DataFrame({'ID':a['ID']})
    for res in res_array:
        r_n = res.df().reset_index()
        r_all = r_all.merge(r_n, on = 'ID')
    return r_all

def download_port (tgt = None):
    global global_mat #Matriz de datos sin agrupar
    global port_dates_list #Vector de fechas de carteras cargadas en PRTU 
    
    #Se inicializa la matriz global que contendrá todos los datos
    global_mat = pd.DataFrame(data = {})
    
    list_port_names = port.value
    for i in range(len(list_port_names)):
        
        if i==0:
            obj = bqport.load_portfolio(list_port_names[i])
            global_mat = obj.to_dataframe().reset_index()
#             global_mat['Banco'] = list_port_names[i]
            aux_bank_name = list_port_names[i].split('-')
            aux_bank_name = aux_bank_name[-1]
            global_mat['Banco'] = aux_bank_name 
            
        else:
            obj = bqport.load_portfolio(list_port_names[i])
            aux_global_mat = obj.to_dataframe().reset_index()
#             aux_global_mat['Banco'] = list_port_names[i]            
            aux_bank_name = list_port_names[i].split('-')
            aux_bank_name = aux_bank_name[-1]
            aux_global_mat['Banco'] = aux_bank_name             
            
            global_mat = pd.concat([global_mat, aux_global_mat])

        i += 1
    
    port_dates_list = sorted(list(global_mat['date'].unique()), reverse = True) #Ordenado de fecha más reciente a la más antigua
    
    #Cargar carteras en el Autocomplete de las Carteras Manejadas
    port_patri.data = global_mat['Banco'].unique().tolist()
    
    #Cargar carteras en el Dropdown de los Depósitos y Retiros
    port_depo_ret.options = global_mat['Banco'].unique().tolist()
    
def download_main_data (tgt = None):
    global global_mat #Matriz de datos sin agrupar
    global agg_global_data_mat
    global list_unique_clasi
    global port_dates_list #Vector de fechas de carteras cargadas en PRTU 
    global def_mat_div
    global def_mat_cup
    
    #Cargar valores en las siguientes variables donde hay widgets
    global assets_class
    
    request_button2.disabled = True #Durante el proceso de esta descarga de datos, se desabilita el botón para evitar errores de doble click    
    wait_msg.layout.visibility = 'visible'
    
    print_message_box.value = 'Descargando cartera(s) ...'
    
    #Descargar cartera(s)
    download_port()
    
    print_message_box.value = 'Descargando caracteriticas ...'
    
    #Filtrar la global_mat por las fechas indicadas en los DatePickers
    list_first_dates = global_mat['date'].loc[global_mat['date'].apply(lambda x: False if x > datetime.datetime.strptime(date_st.value, '%d-%m-%Y') else True)].unique()
    
    try:
        first_date_n_prev = list_first_dates[-1]
    except:
        first_date_n_prev = port_dates_list[-1]        
    
    #Se filtra por el límite inferior de la fecha inicial introducida
    global_mat = global_mat.loc[global_mat['date'] >= first_date_n_prev]
    
    #Se filtra por el límite superior de la fecha final introducida
    global_mat = global_mat.loc[global_mat['date'] <= datetime.datetime.strptime(date_end.value, '%d-%m-%Y')]
    
    
    #Obtener el listado de los activos
    sec_list = list(global_mat['security'].unique())
    
    #La descarga va a haber que hacerla por partes, ya que para los bonos y las divisas los campos FUND_ASSET_CLASS_FOCUS y COUNTRY_FULL_NAME dan problemas

    #Parche para corregir los espacios que se introducen en los tickers de bonos
    sec_list_fi = [] 
    sec_list_eq = []
    for i in range(len(sec_list)):
        if (('Corp' in sec_list[i])==True) or (('Govt' in sec_list[i])==True) or (('Mtge' in sec_list[i])==True) or (('Pfd' in sec_list[i])==True):
            sec_list_fi += [sec_list[i]]

        elif (('Curncy' in sec_list[i])==True):
            sec_list_fi += [sec_list[i]]

        else:
            sec_list_eq += [sec_list[i]]
            
    try:
        aux_mat = download_equity_features(sec_list_eq)        
    
    except:
        aux_mat = pd.DataFrame(columns = ['security', 'País_Assets', 'Sector', 'Tipo_sec', 'Industry_group', 'País_Fondo', 'Name', 'ID_BBG', 'Tipo_fondo', 'ISIN','Maturity_Band_Focus'])
        
    try:
        aux_mat2 = download_fi_or_cur_features(sec_list_fi)
    
    except:
        aux_mat2 = pd.DataFrame(columns = ['País_Assets', 'Sector', 'Matur_type', 'Tipo_sec', 'Cupón', 'Coupon_freq', 'Name', 'Vencimiento', 'Coupon_nxt_date', 'Rating', 'Interests', 'security', 'ISIN'])
    
    #Agrupación de ambas matrices
    aux_mat = pd.concat([aux_mat, aux_mat2]).reset_index().drop(['index'], axis=1)
    
    #Se adjunta esta matriz de datos a la matriz descargada con la serie temporal de las carteras 
    global_mat = pd.merge(global_mat, aux_mat, how='left', on=['security', 'security'])
    global_mat = global_mat.rename(columns = {'currency': 'Divisa', 'quantity': 'Nominal', 'Name': 'Título'})
    
    #Descargar los precios necesarios para la generación del reporting de los activos del portfolio
    download_price_global_mat()
    
    #Parametrización de tipologías de activos
    data_clasification()
    
    #Parametrización del sector s/ la tipología del activo
    adaptacion_sectorial()
    
    #Generar la matriz global del apartado del reporting "Desglose" 
    agg_global_data_mat = pd.DataFrame(data = {})
    for bank in list(global_mat['Banco'].unique()):
        first_date = sorted(list(global_mat.loc[global_mat['Banco'].isin([bank])]['date'].unique()), reverse = True)[0] #Coger la fecha de la cartera más reciente
        agg_global_data_mat = agg_global_data_mat.append(global_mat.loc[global_mat['Banco'].isin([bank]) & global_mat['date'].isin([first_date])], ignore_index=True)
     
    #Agregar a la matriz agg_global_data_mat los campos de país, precio de compra inicial y precio de compra medio ponderado
    compute_country_date_buy_and_buyprice()
    
    #Calcular cuales son las posiciones (Tactical/Candidate) / Core
#     try:
#         TacticalOrCore()
#     except:
#         agg_global_data_mat['Core/Tactical'] = np.nan
        
    print_message_box.value = 'Generando matrices de dividendos y cupones ...'      
    
    #Calcular dividendos
    try:
        dividend_matrix()
        
    except:
        agg_global_data_mat['Dividendos recibidos'] = np.nan
        def_mat_div = pd.DataFrame(columns = ['security'] + [str(i) for i in range(1,13)])
        
    #Calcular cupones
    try:
        coupon_matrix()
        
    except:
        agg_global_data_mat['Intereses recibidos'] = np.nan
        def_mat_cup = pd.DataFrame(columns = ['security'] + [str(i) for i in range(1,13)])
    
    try:
        forward_coupon_matrix()
    except:
        pass
    
    #Ordenar matriz
    agg_global_data_mat = agg_global_data_mat[['Clasificación', 'Banco', 'security', 'ID_BBG', 'Título', 'Divisa', 'Rating', 'País', 'Sector', 'Nominal', 'Cupón', 'Vencimiento', 'Fecha Compra',
                                               'Precio compra', 'Precio compra Report', 'P0_Report', 'Precio Actual', 'P1_Report', 'Dividendos recibidos', 'Intereses recibidos', 'Dividend yield fecha compra',
                                               'Interests', 'ISIN','Maturity_Band_Focus']]
    
    agg_global_data_mat = agg_global_data_mat.drop_duplicates().reset_index()
    agg_global_data_mat = agg_global_data_mat.drop(['index'], axis=1)
    
    #Generar el Excel de Precios
    Load_Prices_in_Excel()
    
    #Cargar valores en widgets
    port_patri.data = global_mat['Banco'].unique().tolist()
    port_depo_ret.options = global_mat['Banco'].unique().tolist()    
    
    list_unique_clasi = list(agg_global_data_mat['Clasificación'].unique())
    assets_class.options = list_unique_clasi
    assets_class.value = list_unique_clasi[0]
    
    ################################################
    #Se sobreescriben los tickers de los bonos en el Excel CHECK_RF, donde se pueden ver los datos de los precios calculados según CBBT y BVAL  
    try:
        list_sec_fi_agg_mat = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Bonos'])]['security'].to_frame()

        book = load_workbook('CHECK_RF.xlsx')
        writer = pd.ExcelWriter('CHECK_RF.xlsx', engine='openpyxl') 
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

        #Limpiar valores en CHECK_RF.xlsx
        clear_df = pd.DataFrame([[None]]*50)
        clear_df.to_excel(writer, 'Soporte')
        writer.save()

        #Guardar nuevos valores en el Excel
        list_sec_fi_agg_mat.to_excel(writer, 'Soporte')
        writer.save()
    
    except:
        pass
    
    ################################################
    
    print_message_box.value = ''
    request_button2.disabled = False #Una vez realizados los cálculos, se vuelve a habilitar el botón
    
    wait_msg.layout.visibility = 'hidden'
    hidden_widgets2.layout.visibility = 'visible'  
    
    
def download_equity_features(equity_asset_list):
    #Petición datos para todos aquellos valores que NO sean ni bonos ni divisas

    #Solicitud de datos
    request =  bql.Request(u.list(equity_asset_list), {'Name': d.name(), 'Tipo_sec': d.SECURITY_TYP()['value'], 'Tipo_fondo': d.FUND_ASSET_CLASS_FOCUS()['value'],
                                                       'País_Assets': f.if_(d.COUNTRY_FULL_NAME()['value'] == 'U.S.', 'UNITED STATES', d.COUNTRY_FULL_NAME()['value']),
                                                       'País_Fondo': f.if_(d.FUND_GEO_FOCUS()['value'] == 'U.S.', 'UNITED STATES', d.FUND_GEO_FOCUS()['value']),
                                                       'Sector': f.if_(d.GICS_SECTOR_NAME()['value'] == 'Financial', 'Financials', d.GICS_SECTOR_NAME()['value']),
                                                       'Industry_group': d.INDUSTRY_GROUP()['value'],
                                                       'ID_BBG': d.ID_BB_COMPANY()['value'], 'ISIN': d.ID_ISIN()['value'],
                                                       'Maturity_Band_Focus':d.FUND_MATURITY_BAND_FOCUS()})
    res = bq.execute(request)
    aux_mat = bq_res_array_to_pd(res)
    aux_mat = aux_mat.rename(columns = {'ID': 'security'})
    
    return aux_mat

def download_fi_or_cur_features(ficur_asset_list):
    #Petición datos para todos aquellos valores que sean bonos o divisas
    
    #Parche para corregir los espacios que se introducen en los tickers de bonos
    aux_ficur_asset_list = []
    for i in range(len(ficur_asset_list)):
        
        if (('Corp' in ficur_asset_list[i])==True) or (('Govt' in ficur_asset_list[i])==True) or (('Mtge' in ficur_asset_list[i])==True) or (('Pfd' in ficur_asset_list[i])==True):
            aux_sec_list = ficur_asset_list[i].split()
            aux_ficur_asset_list += [aux_sec_list[0] + ' ' + aux_sec_list[1]]
            
        else:
            aux_ficur_asset_list += [ficur_asset_list[i]]
    

    #Solicitud de datos
    #Asimismo, para bonos es mejor aplicar el campo BICS_LEVEL_1_SECTOR_NAME en vez del GICS_SECTOR_NAME
    request =  bql.Request(u.list(aux_ficur_asset_list), {'Name': d.LONG_COMP_NAME(), 'Tipo_sec': d.SECURITY_TYP()['value'],
                                                          'País_Assets': f.if_(d.COUNTRY_FULL_NAME()['value'] == 'U.S.', 'UNITED STATES', d.COUNTRY_FULL_NAME()['value']),
                                                          'Sector': f.if_(d.BICS_LEVEL_1_SECTOR_NAME()['value'] == 'Financial', 'Financials', d.BICS_LEVEL_1_SECTOR_NAME()['value']),
                                                          'Interests': d.INT_ACC()['value']/100,
                                                          'Cupón': d.CPN()['value']/100, 'Coupon_nxt_date': d.NXT_CPN_DT()['value'], 'Coupon_freq': d.CPN_FREQ()['value'],
                                                          'Vencimiento': d.MATURITY()['value'], 'Matur_type': d.MTY_TYP()['value'],
                                                          'Rating': f.if_(f.or_(d.RTG_SP()['value'] == np.nan, d.RTG_SP()['value'] == 'N.A.'), d.BB_COMPOSITE()['value'], d.RTG_SP()['value']), 'ISIN': d.ID_ISIN()['value']})
    
    res = bq.execute(request)
    aux_mat2 = bq_res_array_to_pd(res)
    aux_mat2 = aux_mat2.drop(['ID'], axis = 1)
    aux_mat2['security'] = ficur_asset_list
    
    return aux_mat2

def download_price_global_mat (tgt = None):
    global global_mat
    
    #Se descargan los precios necesarios para los cálculos
    list_P1_Base = []
    list_P1_Report = []
    list_P0_Report = []
    for i in range(len(global_mat)):
                
        #Si es un activo de Renta Fija, se descarga el px_bid
        if (('Corp' in global_mat.loc[i]['security'])==True) or (('Govt' in global_mat.loc[i]['security'])==True) or (('Mtge' in global_mat.loc[i]['security'])==True) or (('Pfd' in global_mat.loc[i]['security'])==True) or (global_mat.loc[i]['Sector'] == 'Government'):
        
            request =  bql.Request(u.list(global_mat.loc[i]['security']), {'Precio Actual': d.px_bid(end = datetime.datetime.strptime(date_end.value, '%d-%m-%Y'), fill = 'prev')['value'], 
                                                                           'P1_Report': d.px_bid(end = datetime.datetime.strptime(date_end.value, '%d-%m-%Y'), fill = 'prev', currency = ac_curncy.value)['value'],
                                                                           'P0_Report': d.px_bid(start = global_mat.loc[i]['date'], fill = 'prev', currency = ac_curncy.value)['value']})
            
        elif ('Curncy' in global_mat.loc[i]['security'])==True:
            request =  bql.Request(u.list(global_mat.loc[i]['security']), {'Precio Actual': d.px_last(end = datetime.datetime.strptime(date_end.value, '%d-%m-%Y'), fill = 'prev')['value'], 
                                                                           'P1_Report': f.div(1, d.px_last(end = datetime.datetime.strptime(date_end.value, '%d-%m-%Y'), fill = 'prev', currency = ac_curncy.value))['value'],
                                                                           'P0_Report': f.div(1, d.px_last(start = global_mat.loc[i]['date'], fill = 'prev', currency = ac_curncy.value))['value']})
            
        else:
            request =  bql.Request(u.list(global_mat.loc[i]['security']), {'Precio Actual': d.px_last(end = datetime.datetime.strptime(date_end.value, '%d-%m-%Y'), fill = 'prev')['value'], 
                                                                           'P1_Report': d.px_last(end = datetime.datetime.strptime(date_end.value, '%d-%m-%Y'), fill = 'prev', currency = ac_curncy.value)['value'],
                                                                           'P0_Report': d.px_last(start = global_mat.loc[i]['date'], fill = 'prev', currency = ac_curncy.value)['value']})            
            
        res = bq.execute(request)
        
        aux_p1_base = bq_res_array_to_pd(res)['Precio Actual'][0]
        aux_p1_report = bq_res_array_to_pd(res)['P1_Report'][0]
        aux_p0_report = bq_res_array_to_pd(res)['P0_Report'][0]

        if global_mat.loc[i]['Divisa'] != ac_curncy.value and aux_p1_base == aux_p1_report:
            request =  bql.Request(ac_curncy.value + global_mat.loc[i]['Divisa'] + ' Curncy', {'FX': d.px_last(end = datetime.datetime.strptime(date_end.value, '%d-%m-%Y'), fill = 'prev')['value']})
            res = bq.execute(request)
            fx_df = res[0].df()

            aux_p1_report = aux_p1_report / fx_df['FX'][0]
            
            request =  bql.Request(ac_curncy.value + global_mat.loc[i]['Divisa'] + ' Curncy', {'FX': d.px_last(start = global_mat.loc[i]['date'], fill = 'prev')['value']})
            res = bq.execute(request)
            fx_df = res[0].df()

            aux_p0_report = aux_p0_report / fx_df['FX'][0]            
        
        list_P1_Base += [aux_p1_base]
        list_P1_Report += [aux_p1_report]
        list_P0_Report += [aux_p0_report]

    global_mat['Precio Actual'] = list_P1_Base
    global_mat['P1_Report'] = list_P1_Report
    
    #Si la divisa de un activo está en la que se reporta, entonces el precio 'P0_Report' tiene que ser el mismo del campo 'price' 
    for i in range(len(global_mat)):
        if global_mat.loc[i]['Divisa'] == ac_curncy.value:
            list_P0_Report[i] = global_mat.loc[i]['price']
    
    global_mat['P0_Report'] = list_P0_Report

def data_clasification (tgt = None):
    global global_mat

    global_mat.loc[global_mat['security'].str.contains('Curncy'), 'Clasificación'] = 'CASH'
    global_mat.loc[(global_mat['Tipo_sec'].str.contains('Common Stock')) & (global_mat['security'].str.contains('Equity')), 'Clasificación'] = 'Acciones'
    global_mat.loc[(global_mat['Tipo_sec'].str.contains('ADR')) & (global_mat['security'].str.contains('Equity')), 'Clasificación'] = 'Acciones'
    global_mat.loc[global_mat['security'].str.contains('Corp'), 'Clasificación'] = 'Bonos'
    global_mat.loc[global_mat['security'].str.contains('Mtge'), 'Clasificación'] = 'Bonos'
    global_mat.loc[global_mat['security'].str.contains('Govt'), 'Clasificación'] = 'Tesoros'
    global_mat.loc[global_mat['security'].str.contains(' Pfd'), 'Clasificación'] = 'Preferentes' 
    global_mat.loc[global_mat['Sector'] == 'Government', 'Clasificación'] = 'Tesoros'  
    global_mat.loc[(global_mat['Tipo_sec'].str.contains('ETP')) & (global_mat['Tipo_fondo'].str.contains('Equity')) & (global_mat['security'].str.contains('Equity')), 'Clasificación'] = 'ETFs RV'
    global_mat.loc[(global_mat['Tipo_sec'].str.contains('ETP')) & (global_mat['Industry_group'].str.contains('Equity Fund')) & (global_mat['security'].str.contains('Equity')), 'Clasificación'] = 'ETFs RV'
    global_mat.loc[(global_mat['Tipo_sec'].str.contains('ETP')) & (global_mat['Tipo_fondo'].str.contains('Fixed Income')) & (global_mat['security'].str.contains('Equity')), 'Clasificación'] = 'ETFs RF'
    global_mat.loc[(global_mat['Tipo_sec'].str.contains('ETP')) & (global_mat['Industry_group'].str.contains('Debt Fund')) & (global_mat['security'].str.contains('Equity')), 'Clasificación'] = 'ETFs RF'
    global_mat.loc[(global_mat['Tipo_sec'].str.contains('Open-End Fund')) & (global_mat['Tipo_fondo'].str.contains('Equity')) & (global_mat['security'].str.contains('Equity')), 'Clasificación'] = 'Fondos RV'
    global_mat.loc[(global_mat['Tipo_sec'].str.contains('Fund of Funds')) & (global_mat['Tipo_fondo'].str.contains('Equity')) & (global_mat['security'].str.contains('Equity')), 'Clasificación'] = 'Fondos RV'
    global_mat.loc[(global_mat['Tipo_sec'].str.contains('Open-End Fund')) & (global_mat['Industry_group'].str.contains('Equity Fund')) & (global_mat['security'].str.contains('Equity')), 'Clasificación'] = 'Fondos RV'
    global_mat.loc[(global_mat['Tipo_sec'].str.contains('Fund of Funds')) & (global_mat['Industry_group'].str.contains('Equity Fund')) & (global_mat['security'].str.contains('Equity')), 'Clasificación'] = 'Fondos RV'
    global_mat.loc[(global_mat['Tipo_sec'].str.contains('Open-End Fund')) & (global_mat['Tipo_fondo'].str.contains('Fixed Income')) & (global_mat['security'].str.contains('Equity')), 'Clasificación'] = 'Fondos RF'
    global_mat.loc[(global_mat['Tipo_sec'].str.contains('Fund of Funds')) & (global_mat['Tipo_fondo'].str.contains('Fixed Income')) & (global_mat['security'].str.contains('Equity')), 'Clasificación'] = 'Fondos RF'
    global_mat.loc[(global_mat['Tipo_sec'].str.contains('Open-End Fund')) & (global_mat['Industry_group'].str.contains('Debt Fund')) & (global_mat['security'].str.contains('Equity')), 'Clasificación'] = 'Fondos RF'
    global_mat.loc[(global_mat['Tipo_sec'].str.contains('Fund of Funds')) & (global_mat['Industry_group'].str.contains('Debt Fund')) & (global_mat['security'].str.contains('Equity')), 'Clasificación'] = 'Fondos RF'
    global_mat['Clasificación'] = global_mat['Clasificación'].fillna('Sin Clasificar')    

def compute_country_date_buy_and_buyprice (tgt = None):
    global agg_global_data_mat
    
    # Asignar a cada título su país
    # Se juntan los valores de las columnas País_Assets y País_Fondo en una sola columna llamda País en función de la tipología del activo
    for clasi in list(global_mat['Clasificación'].unique()):
        if clasi == 'Acciones' or clasi == 'Bonos' or clasi == 'CASH' or clasi == 'Preferentes' or clasi == 'Tesoros':       
            filtered_mat = global_mat.loc[global_mat['Clasificación'].isin([clasi])][['security', 'País_Assets']].drop_duplicates()
            filtered_mat = filtered_mat.rename(index=str, columns={'País_Assets': 'País'})
        else:
            filtered_mat = global_mat.loc[global_mat['Clasificación'].isin([clasi])][['security', 'País_Fondo']].drop_duplicates()
            filtered_mat = filtered_mat.rename(index=str, columns={'País_Fondo': 'País'})

        if clasi == list(global_mat['Clasificación'].unique())[0]:
            agg_global_data_mat = pd.merge(agg_global_data_mat, filtered_mat,  how='left', on=['security', 'security'])
        else:
            aux_agg_global_data_mat = pd.merge(agg_global_data_mat.loc[:, agg_global_data_mat.columns != 'País'], filtered_mat,  how='left', on=['security', 'security'])
            agg_global_data_mat['País'] = agg_global_data_mat['País'].fillna(value = aux_agg_global_data_mat['País'])

    #Buscar Fecha de compra inicial        
    list_dates = []
    for found_bank, found_asset in zip(agg_global_data_mat['Banco'].tolist(), agg_global_data_mat['security'].tolist()):
        aux_list_dates = global_mat.loc[global_mat['Banco'].isin([found_bank]) & global_mat['security'].isin([found_asset])].sort_values(by=['date'], ascending = True)['date'].tolist()
        list_dates += [aux_list_dates[0]]

    agg_global_data_mat['Fecha Compra'] = list_dates
    
    #calculo el div_per_share de los equities ya que no era un campo dado de alto en global_mat
    list_dvd_yield = []
    global_mat_div_share_aux = global_mat
    for idx in range(len(global_mat_div_share_aux)):
        if 'Equity' in global_mat_div_share_aux.loc[idx]['security']:
            try:
                request = bql.Request(global_mat_div_share_aux.loc[idx]['security'], {'dvd_yld': d.IS_DIV_PER_SHR(as_of_date = global_mat_div_share_aux.loc[idx]['date'].to_pydatetime(),
                                                                                                             fill = 'prev',
                                                                                                             currency = ac_curncy.value)})
                res = bq.execute(request)
                aux_res = res[0].df().reset_index()
                list_dvd_yield += [aux_res['dvd_yld'].values[0]]

            except:
                list_dvd_yield += [np.nan]

        else:
            list_dvd_yield += [np.nan]    

    global_mat_div_share_aux['Dividend yield fecha compra'] = list_dvd_yield
    global_mat_div_share_aux = global_mat_div_share_aux[['date','security','Nominal','Banco','Dividend yield fecha compra']]


    #Div_per_share ponderado por compra
    list_weighted_div_per = []
    for found_bank, found_asset in zip(agg_global_data_mat['Banco'].tolist(), agg_global_data_mat['security'].tolist()):
        #Se genera una matriz de compra de cada activo
        aux_mat = global_mat_div_share_aux.loc[global_mat_div_share_aux[['Banco', 'security']].isin([found_bank, found_asset]).all(axis='columns')][['date', 'Nominal', 'Dividend yield fecha compra']].reset_index()
        aux_mat['Compra_Venta'] = aux_mat['Nominal'].diff()
        aux_mat['Compra_Venta'] = aux_mat['Compra_Venta'].fillna(aux_mat['Nominal'][0])
        aux_mat = aux_mat.loc[aux_mat['Compra_Venta']>0] #Filtras las Compras

        list_weighted_div_per += [sum(aux_mat['Nominal']*aux_mat['Dividend yield fecha compra'])/sum(aux_mat['Nominal'])]

    agg_global_data_mat['Dividend yield fecha compra'] = list_weighted_div_per

    #Precio medio ponderado de compra
    list_weighted_prices = []
    list_weighted_prices_reporting_fx = []
    for found_bank, found_asset in zip(agg_global_data_mat['Banco'].tolist(), agg_global_data_mat['security'].tolist()):
        #Se genera una matriz de compra de cada activo
        aux_mat = global_mat.loc[global_mat[['Banco', 'security']].isin([found_bank, found_asset]).all(axis='columns')][['date', 'Nominal', 'price', 'P0_Report']].reset_index()
        aux_mat['Compra_Venta'] = aux_mat['Nominal'].diff()
        aux_mat['Compra_Venta'] = aux_mat['Compra_Venta'].fillna(aux_mat['Nominal'][0])
        aux_mat = aux_mat.loc[aux_mat['Compra_Venta']>0] #Filtras las Compras

        list_weighted_prices += [sum(aux_mat['Nominal']*aux_mat['price'])/sum(aux_mat['Nominal'])]
        list_weighted_prices_reporting_fx += [sum(aux_mat['Nominal']*aux_mat['P0_Report'])/sum(aux_mat['Nominal'])]
    
    agg_global_data_mat['Precio compra'] = list_weighted_prices
    agg_global_data_mat['Precio compra Report'] = list_weighted_prices_reporting_fx
    
# def TacticalOrCore (tgt = None):
#     global agg_global_data_mat

#     list_eq = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Acciones']), 'security'].tolist()
#     list_id_bbg = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Acciones']), 'ID_BBG'].tolist()

#     df = pd.DataFrame(data= {'ID': list_eq, 'ID_BBG': list_id_bbg})
#     res_df = pd.DataFrame(data={'Core/Tactical': [], 'ID_BBG': []})

#     for eq in list_eq:
#         try:

#             univ1 = u.list(eq)
#             univ1 = u.translatesymbols(univ1, TARGETIDTYPE="FUNDAMENTALTICKER")

#             univ2 = u.members(['SPDAUDT Index', 'SPDAEEP Index', 'SPGDAUP Index', 'SREITTGL Index'])
#             univ2 = u.translatesymbols(univ2, TARGETIDTYPE="FUNDAMENTALTICKER")

#             ids_in_idx = u.intersect(univ1, univ2)
#             req = bql.Request(ids_in_idx, {'Core/Tactical': d.id()['value'], 'ID_BBG': d.ID_BB_COMPANY()['value']})
#             res = bq.execute(req)
#             aux_res_df = bq_res_array_to_pd(res)
#             aux_res_df = aux_res_df.drop(['ID'], axis = 1)
#             res_df = res_df.append(aux_res_df, ignore_index = True) 

#         except:
#             pass

#     df = pd.merge(df, res_df, how='left', on=['ID_BBG'])
#     df = df.rename(columns = {'ID': 'security'})
#     df = df.drop(['ID_BBG'], axis = 1)
#     df['Core/Tactical'] = df['Core/Tactical'].fillna('Candidate/Tactical')

#     for i_idx in range(len(df)):
#         if df.loc[i_idx, 'Core/Tactical'] != 'Candidate/Tactical':
#             df.loc[i_idx, 'Core/Tactical'] = 'Core' 

#     agg_global_data_mat = pd.merge(agg_global_data_mat, df, how='left', on=['security'])     
    
def adaptacion_sectorial (tgt = None):
    global global_mat
    
    list_def_sec = []
    for idx in range(len(global_mat)):
        if global_mat.loc[idx, 'Clasificación'] == 'ETFs RV' or global_mat.loc[idx, 'Clasificación'] == 'ETFs RF' or global_mat.loc[idx, 'Clasificación'] == 'Fondos RV' or global_mat.loc[idx, 'Clasificación'] == 'Fondos RF':
            list_def_sec += [global_mat.loc[idx, 'Industry_group']]
        else:
            list_def_sec += [global_mat.loc[idx, 'Sector']]
    list_def_sec

    global_mat['aux_Sector'] = list_def_sec
    global_mat = global_mat.drop(['Sector'], axis=1)    
    global_mat = global_mat.rename(columns = {'aux_Sector': 'Sector'})
    
def gen_carteras_manejadas(tgt = None):
    global df_bancos_manejada
    
    #Ver si existe df_bancos_manejada, si no existe crearla, si existe añadir el valor
    try:
        df_bancos_manejada.columns
        df_bancos_manejada = df_bancos_manejada.append(pd.DataFrame(data = {'Banco': port_patri.value, 'Fecha Inicial': float(patri_st_date.value), 'Fecha Final': float(patri_end_date.value), 'Tipo': 'Manejada'}, index = [0]), ignore_index=True)
        
    except:
        df_bancos_manejada = pd.DataFrame(data = {'Banco': port_patri.value, 'Fecha Inicial': float(patri_st_date.value), 'Fecha Final': float(patri_end_date.value), 'Tipo': 'Manejada'}, index = [0])
        
    print_message_box.value = 'Añadida cartera manejada'
    
def generate_dep_ret(tgt = None):
    global dep_ret_mat
    
    #Ver si existe dep_ret_mat, si no existe crearla, si existe añadir el valor
    try:
        dep_ret_mat.columns
        dep_ret_mat = dep_ret_mat.append(pd.DataFrame(data={'Banco': port_depo_ret.value, 'Valor': float(depo_ret.value)}, index = [0]), ignore_index=True)
        
    except:
        dep_ret_mat = pd.DataFrame(data = {'Banco': port_depo_ret.value, 'Valor': float(depo_ret.value)}, index = [0])
        
    print_message_box.value = 'Añadido depósito/retiro'
    
def dividend_matrix(tgt = None): 
    global global_mat
    global agg_global_data_mat
    
    global def_mat_div #Aquí se genera la matriz de dividendos para el reporting
    global tot_div    
    
    dividends_df = pd.DataFrame(data={})

    #Aplicar los cálculos unicamente sobre los tickers que contengan la palabra Equity
    list_eq_tickers = []
    for eq in global_mat['security'].unique():
        if 'Equity' in eq:
            list_eq_tickers += [eq]

    for eq in list_eq_tickers:
        i_start_date = datetime.datetime.strptime(date_st.value, '%d-%m-%Y')
        i_end_date = datetime.datetime.strptime(date_end.value, '%d-%m-%Y')

        try:
            request = bql.Request(eq, {'Dividendos': f.dropna(d.cash_divs(dates = f.range(i_start_date, i_end_date), ca_date_type = 'PAY_DATE', currency = ac_curncy.value))})
            res = bq.execute(request)
            aux_res = res[0].df().reset_index()
            aux_res['aux'] = range(len(aux_res)) #Se Asigna una variable auxiliar para identificar dónde se tienen que colocar las fechas ex-dividend

            #Para buscar las fechas Ex-Dividend, se va a coger el último valor del dividendo pagado y al primero se le resta 1 año.
            #No obstante, en vez de sacar el histórico, sacamos el número mínimo necesario de las fechas ex-dividends que necesitamos con la función last
            #Se alinean los valores con la variable auxiliar creada al hacer un merge

            if len(aux_res) > 1:
                aux_st_dt = aux_res['DATE'].values[0]
                aux_end_dt = aux_res['DATE'].values[-1]

            else:
                aux_st_dt = aux_res['DATE'].values[0]
                aux_end_dt = aux_res['DATE'].values[0]

            pay_st_dt = pd.to_datetime(str(aux_st_dt - np.timedelta64(8760, 'h'))).strftime('%Y-%m-%d') 
            pay_end_dt = pd.to_datetime(str(aux_end_dt)).strftime('%Y-%m-%d')

            request = bql.Request(eq, {'Dividendos': f.last(f.dropna(d.cash_divs(dates = f.range(pay_st_dt, pay_end_dt), ca_date_type = 'EFFECTIVE_DATE', currency = ac_curncy.value)), n = len(aux_res))})
            res = bq.execute(request)
            aux_res2 = res[0].df().reset_index().rename(index=str, columns={'DATE': 'Ex-div_date'})
            aux_res2['aux'] = range(len(aux_res2))

            aux_res = pd.merge(aux_res, aux_res2[['aux', 'Ex-div_date']], how = 'left', on = ['aux'])

            dividends_df = dividends_df.append(aux_res, ignore_index = True)

        except:
            pass

    dividends_df = dividends_df.dropna(subset=['Dividendos'])     
    
    df_tick_banks = global_mat[['Banco', 'security']].drop_duplicates()
    df_tick_banks = df_tick_banks.rename(index=str, columns={'security': 'ID'}) 

    dividends_df = pd.merge(dividends_df, df_tick_banks, how='left', on=['ID'])    

    #Se genera una pivot table donde las filas son los activos y las cantidas y las columnas son los periodos temporales que se han cargado
    n_assets = pd.DataFrame(data={})

    for i_bank in global_mat['Banco'].unique():
        aux_i_global_mat = global_mat.loc[global_mat['Banco'].isin([i_bank])]
        n_assets = pd.concat([n_assets, aux_i_global_mat[['date', 'Banco', 'security', 'Nominal']].pivot_table(index = ['Banco', 'security'], columns = 'date', values = 'Nominal').fillna(0).reset_index()], ignore_index=True)

    list_dates_concat = []
    for i_dates_concat in n_assets.columns:
        if i_dates_concat != 'Banco' and i_dates_concat != 'security':
            list_dates_concat += [i_dates_concat]

    n_assets = n_assets[['Banco', 'security'] + sorted(list_dates_concat)]
    n_assets = n_assets.fillna(method='ffill', axis = 1)

    list_amount_div = []
    for i in range(len(dividends_df)):
        aux_mat = n_assets.loc[n_assets['security'].isin([dividends_df['ID'].iloc[i]]) & n_assets['Banco'].isin([dividends_df['Banco'].iloc[i]])].drop(['Banco', 'security'], axis=1) 

        try:
            if aux_mat.loc[:,aux_mat.columns <= dividends_df['Ex-div_date'].iloc[i]].iloc[:,-1].values[0] != 0: #Si no se ha comprado el valor antes de la fecha ex-dividend, entonces no se podrá computar el dividendo recibido en la fecha de su pago            
                date_quantity = aux_mat.loc[:,aux_mat.columns <= dividends_df['Ex-div_date'].iloc[i]].iloc[:,-1].values[0]
                list_amount_div += [date_quantity * dividends_df['Dividendos'].iloc[i]]

            else:
                list_amount_div += [0]

        except: #Si no hay nada comprado para el activo en el que en esa fecha se paga un dividendo
            list_amount_div += [0]

    dividends_df['Total_Div'] = pd.Series(list_amount_div).apply(pd.to_numeric, errors='coerce').fillna(0)         
    dividends_df['mes'] = dividends_df['DATE'].map(lambda x: x.month)
    dividends_df['año'] = dividends_df['DATE'].map(lambda x: x.year)

    #Generar la matriz de dividendos del último año
    last_year = np.sort(dividends_df['año'].unique())[-1]
    fil_dividends_df = dividends_df.loc[dividends_df['año'].isin([last_year])]

    cum_dividends_df = pd.pivot_table(fil_dividends_df, values='Total_Div', index=['ID'],columns=['mes'], aggfunc=np.sum)
    tot_div = cum_dividends_df.sum(axis=1,skipna=True).sum(axis=0,skipna=True)

    def_mat_div = pd.DataFrame(data={})
    for col in cum_dividends_df.columns.tolist():
        def_mat_div[str(col)] = cum_dividends_df[col].tolist()

    if cum_dividends_df.columns.tolist() != list(range(1,13)): #Ver si faltan meses por incluir en el año
        for left_month in list(set(list(range(1,13))) - set(cum_dividends_df.columns.tolist())):
            def_mat_div[str(left_month)] = np.nan

    def_mat_div['security'] = cum_dividends_df.index.tolist() 
    
#     def_mat_div = def_mat_div[['Ticker'] + [str(i) for i in range(1,13)]] #Ordenar la matriz
#     def_mat_div['Total'] = def_mat_div.sum(axis=1,skipna=True)
#     def_mat_div = def_mat_div.append(def_mat_div.sum(numeric_only=True), ignore_index=True)
#     def_mat_div.loc[len(def_mat_div)-1, 'Ticker'] = 'Total'
#     def_mat_div = def_mat_div.loc[def_mat_div['Total'] != 0]

    #Una vez calucalda la matriz de dividendos que servirá para el Global Reporting, generar el df para concatenar la cantidad de dividendos recibidos por ticker y banco
    new_mat_div = pd.pivot_table(dividends_df, values='Total_Div', index=['ID', 'Banco'],columns=['mes'], aggfunc=np.sum)

    new_mat_div['Dividendos recibidos'] = new_mat_div.sum(axis=1,skipna=True)
    new_mat_div = new_mat_div.reset_index()
    new_mat_div = new_mat_div.rename(index=str, columns={'ID': 'security'})
    new_mat_div = new_mat_div[['security', 'Banco', 'Dividendos recibidos']]

    agg_global_data_mat = pd.merge(agg_global_data_mat, new_mat_div, how='left', on=['security', 'Banco'])  
    
    #Calcular el dividend yield

def coupon_matrix(tgt = None):
    global global_mat
    global agg_global_data_mat
    
    global def_mat_cup #Aquí se genera la matriz de cupones para el reporting
    global tot_cup
    
    # Cálculo matriz cupones recibidos

   #Se intenta generar un DataFrame como el que devolvía la query de los dividendos recibidos por activo
    coupons_df = pd.DataFrame(data={})

    aux_coupons_df = global_mat.loc[global_mat['Clasificación'].isin(['Bonos','Tesoros'])][['Banco', 'security', 'Coupon_freq', 'Coupon_nxt_date', 'Cupón', 'Divisa']].drop_duplicates()

    start_dt = datetime.datetime.strptime(date_st.value, '%d-%m-%Y')
    end_dt = datetime.datetime.strptime(date_end.value, '%d-%m-%Y')

    for i_idx in aux_coupons_df.index:
        try:
            i_sec_nxt_cup = aux_coupons_df.loc[i_idx, 'Coupon_nxt_date'].to_pydatetime()
            i_sec_cup = aux_coupons_df.loc[i_idx, 'Cupón']
            i_sec_cup_feq = aux_coupons_df.loc[i_idx, 'Coupon_freq']
            i_sec_cup = i_sec_cup/i_sec_cup_feq
            i_sec_cup_feq = 12/i_sec_cup_feq

            list_cup_dates = []
            time_count = i_sec_nxt_cup
            while start_dt <= time_count:
                if time_count <= end_dt:
                    list_cup_dates += [time_count]

                time_count -= datetime.timedelta(i_sec_cup_feq*365/12)

            if len(list_cup_dates) > 0:
                i_sec_coupons_df = pd.DataFrame(data={'security': aux_coupons_df.loc[i_idx, 'security'],
                                                      'Coupon_Payment_Dt': list_cup_dates, 'Cupón': i_sec_cup,
                                                      'Divisa': aux_coupons_df.loc[i_idx, 'Divisa'],
                                                      'Banco': aux_coupons_df.loc[i_idx, 'Banco']})
                coupons_df = coupons_df.append(i_sec_coupons_df, ignore_index=True)

        except:
            continue

    coupons_df = coupons_df.drop_duplicates() 

     #Una vez generado el DataFrame de forma similar al los dividendos recibidos, el resto de los cálculos se realizan de forma análoga

    #Se genera una pivot table donde las filas son los activos y las cantidas y las columnas son los periodos temporales que se han cargado
    n_assets = pd.DataFrame(data={})

    for i_bank in global_mat['Banco'].unique():
        aux_i_global_mat = global_mat.loc[global_mat['Banco'].isin([i_bank])]
        n_assets = pd.concat([n_assets, aux_i_global_mat[['date', 'Banco', 'security', 'Nominal']].pivot_table(index = ['Banco', 'security'], columns = 'date', values = 'Nominal').fillna(0).reset_index()], ignore_index=True)

    list_dates_concat = []
    for i_dates_concat in n_assets.columns:
        if i_dates_concat != 'Banco' and i_dates_concat != 'security':
            list_dates_concat += [i_dates_concat]

    n_assets = n_assets[['Banco', 'security'] + sorted(list_dates_concat)]
    n_assets = n_assets.fillna(method='ffill', axis = 1)

    list_amount_cup = []
    for i in range(len(coupons_df)):
        aux_mat = n_assets.loc[n_assets['security'].isin([coupons_df['security'].iloc[i]]) & n_assets['Banco'].isin([coupons_df['Banco'].iloc[i]])].drop(['Banco', 'security'], axis=1) 

        try:
            date_quantity = aux_mat.loc[:,aux_mat.columns <= coupons_df['Coupon_Payment_Dt'].iloc[i]].iloc[:,-1].values[0]
            list_amount_cup += [date_quantity * coupons_df['Cupón'].iloc[i]]        

        except: #Si no hay nada comprado para el activo en el que en esa fecha se paga un dividendo ***
            list_amount_cup += [0]   


    #Incluir preferentes
    if len(global_mat.loc[global_mat['Clasificación'].isin(['Preferentes'])])>0:

        pref_carry_df = pd.DataFrame(data={})

        pref_list = list(global_mat.loc[global_mat['Clasificación'].isin(['Preferentes'])]['security'].unique())

        for eq in pref_list:
            i_start_date = datetime.datetime.strptime(date_st.value, '%d-%m-%Y')
            i_end_date = datetime.datetime.strptime(date_end.value, '%d-%m-%Y')

            try:
                request = bql.Request(eq, {'Cupón': f.dropna(d.cash_divs(CA_ADJ='RAW', dates = f.range(i_start_date, i_end_date), ca_date_type = 'PAY_DATE', currency = ac_curncy.value))})
                res = bq.execute(request)
                aux_res = res[0].df().reset_index()
                aux_res['aux'] = range(len(aux_res)) #Se Asigna una variable auxiliar para identificar dónde se tienen que colocar las fechas ex-dividend

                #Para buscar las fechas Ex-Dividend, se va a coger el último valor del dividendo pagado y al primero se le resta 1 año.
                #No obstante, en vez de sacar el histórico, sacamos el número mínimo necesario de las fechas ex-dividends que necesitamos con la función last
                #Se alinean los valores con la variable auxiliar creada al hacer un merge

                if len(aux_res) > 1:
                    aux_st_dt = aux_res['DATE'].values[0]
                    aux_end_dt = aux_res['DATE'].values[-1]

                else:
                    aux_st_dt = aux_res['DATE'].values[0]
                    aux_end_dt = aux_res['DATE'].values[0]

                pay_st_dt = pd.to_datetime(str(aux_st_dt - np.timedelta64(8760, 'h'))).strftime('%Y-%m-%d') 
                pay_end_dt = pd.to_datetime(str(aux_end_dt)).strftime('%Y-%m-%d')

                request = bql.Request(eq, {'Cupón': f.last(f.dropna(d.cash_divs(CA_ADJ='RAW', dates = f.range(pay_st_dt, pay_end_dt), ca_date_type = 'EFFECTIVE_DATE', currency = ac_curncy.value)), n = len(aux_res))})
                res = bq.execute(request)
                aux_res2 = res[0].df().reset_index().rename(index=str, columns={'DATE': 'Ex-div_date'})
                aux_res2['aux'] = range(len(aux_res2))

                aux_res = pd.merge(aux_res, aux_res2[['aux', 'Ex-div_date']], how = 'left', on = ['aux'])

                pref_carry_df = pref_carry_df.append(aux_res, ignore_index = True)

            except:
                pass

        pref_carry_df = pref_carry_df.dropna(subset=['Cupón'])
        pref_carry_df = pref_carry_df.drop(['aux'], axis=1)

        df_tick_banks = global_mat[['Banco', 'security']].drop_duplicates()
        df_tick_banks = df_tick_banks.rename(index=str, columns={'security': 'ID'}) 

        pref_carry_df = pd.merge(pref_carry_df, df_tick_banks, how='left', on=['ID'])    

        list_amount_pref = []
        for i in range(len(pref_carry_df)):
            aux_mat = n_assets.loc[n_assets['security'].isin([pref_carry_df['ID'].iloc[i]]) & n_assets['Banco'].isin([pref_carry_df['Banco'].iloc[i]])].drop(['Banco', 'security'], axis=1) 

            try:
                if aux_mat.loc[:,aux_mat.columns <= pref_carry_df['Ex-div_date'].iloc[i]].iloc[:,-1].values[0] != 0: #Si no se ha comprado el valor antes de la fecha ex-dividend, entonces no se podrá computar el dividendo recibido en la fecha de su pago            
                    date_quantity = aux_mat.loc[:,aux_mat.columns <= pref_carry_df['DATE'].iloc[i]].iloc[:,-1].values[0]
                    list_amount_pref += [date_quantity * pref_carry_df['Cupón'].iloc[i]]

                else:
                    list_amount_pref += [0]

            except: #Si no hay nada comprado para el activo en el que en esa fecha se paga el carry del preferente
                list_amount_pref += [0]

        #Adaptar el formato de los pagos de las preferentes (similar al de dividendos) al de los cupones
        pref_carry_df = pref_carry_df.rename(index=str, columns={'ID': 'security', 'DATE': 'Coupon_Payment_Dt', 'CURRENCY': 'Divisa'})

        #Agrupar las preferentes a los cupones        
        coupons_df = coupons_df.append(pref_carry_df[['security', 'Coupon_Payment_Dt', 'Cupón', 'Divisa', 'Banco']], ignore_index=True)    

        list_amount_cup = list_amount_cup + list_amount_pref

    coupons_df['Total_Coupon'] = pd.Series(list_amount_cup).apply(pd.to_numeric, errors='coerce').fillna(0)  
    coupons_df['mes'] = coupons_df['Coupon_Payment_Dt'].map(lambda x: x.month)

    #En caso de que algún bono esté en otra divisa, calcular el cambio de divisa
    list_real_amount_cup = []
    for i_idx in coupons_df.index:
        divisa_activo = coupons_df.loc[i_idx, 'Divisa']
        if divisa_activo != ac_curncy.value:
            request =  bql.Request(ac_curncy.value + divisa_activo + ' Curncy', {'FX': d.px_last(start = coupons_df.loc[i_idx, 'Coupon_Payment_Dt'].to_pydatetime(), fill = 'prev')['value']})
            res = bq.execute(request)
            fx_df = res[0].df()

            list_real_amount_cup += [coupons_df.loc[i_idx, 'Total_Coupon'] / fx_df['FX'][0]]
        else:
            list_real_amount_cup += [coupons_df.loc[i_idx, 'Total_Coupon']]

    coupons_df['Real_Total_Coupon'] = list_real_amount_cup 
    coupons_df['año'] = coupons_df['Coupon_Payment_Dt'].map(lambda x: x.year)

    #Generar la matriz de cupones del último año
    last_year = np.sort(coupons_df['año'].unique())[-1]
    fil_coupons_df = coupons_df.loc[coupons_df['año'].isin([last_year])]

    cum_coupons_df = pd.pivot_table(fil_coupons_df, values='Real_Total_Coupon', index=['security'],columns=['mes'], aggfunc=np.sum)
    tot_cup = cum_coupons_df.apply(pd.to_numeric, errors='coerce').sum().sum()

    def_mat_cup = pd.DataFrame(data={})
    for col in cum_coupons_df.columns.tolist():
        def_mat_cup[str(col)] = cum_coupons_df[col].tolist()

    if cum_coupons_df.columns.tolist() != list(range(1,13)): #Ver si faltan meses por incluir en el año
        for left_month in list(set(list(range(1,13))) - set(cum_coupons_df.columns.tolist())):
            def_mat_cup[str(left_month)] = np.nan

    def_mat_cup['security'] = cum_coupons_df.index.tolist()
    #     def_mat_cup = def_mat_cup[['Ticker'] + [str(i) for i in range(1,13)]] #Ordenar la matriz
    #     def_mat_cup['Total'] = def_mat_cup.sum(axis=1,skipna=True)
    #     def_mat_cup = def_mat_cup.append(def_mat_cup.sum(numeric_only=True), ignore_index=True)
    #     def_mat_cup.loc[len(def_mat_cup)-1, 'Ticker'] = 'Total'

    new_mat_cup = pd.pivot_table(coupons_df, values='Real_Total_Coupon', index=['security', 'Banco'],columns=['mes'], aggfunc=np.sum)

    new_mat_cup['Intereses recibidos'] = new_mat_cup.apply(pd.to_numeric, errors='coerce').sum(axis=1)
    new_mat_cup = new_mat_cup.reset_index()
    new_mat_cup = new_mat_cup[['security', 'Banco', 'Intereses recibidos']]

    agg_global_data_mat = pd.merge(agg_global_data_mat, new_mat_cup, how='left', on=['security', 'Banco'])
        
def forward_coupon_matrix(tgt = None):
    global global_mat
    global agg_global_data_mat
    global aux_coupons_df
    global def_forward_mat_cup #Aquí se genera la matriz de cupones que se recibirán para el reporting    
    
    # Generar la matriz de dividendos futuros será similar a lo anterior, solo que se coge como end_date 12-31-2XXX
    coupons_df = pd.DataFrame(data={})

    aux_coupons_df = pd.merge(agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Bonos','Tesoros'])]['security'].to_frame(), global_mat.loc[global_mat['Clasificación'].isin(['Bonos','Preferentes','Tesoros'])][['Banco', 'security', 'Coupon_freq', 'Coupon_nxt_date', 'Cupón', 'Divisa']].drop_duplicates(), how='left', on=['security', 'security'])

    start_dt = datetime.datetime.strptime(date_st.value, '%d-%m-%Y')
    end_dt = datetime.datetime.strptime(date_end.value, '%d-%m-%Y')

    fut_cup_st_date = datetime.datetime(end_dt.year, 1, 1)
    fut_cup_end_date = datetime.datetime(end_dt.year, 12, 31)

    for i_idx in aux_coupons_df.index:
        try:
            i_sec_nxt_cup = aux_coupons_df.loc[i_idx, 'Coupon_nxt_date'].to_pydatetime()
            i_sec_cup = aux_coupons_df.loc[i_idx, 'Cupón']
            i_sec_cup_feq = aux_coupons_df.loc[i_idx, 'Coupon_freq']
            i_sec_cup = i_sec_cup/i_sec_cup_feq
            i_sec_cup_feq = 12/i_sec_cup_feq

            list_cup_dates = []
            time_count = i_sec_nxt_cup
            while fut_cup_st_date <= time_count:
                if time_count <= fut_cup_end_date:
                    list_cup_dates += [time_count]

                time_count -= datetime.timedelta(i_sec_cup_feq*365/12)

            try:
                #El primer elemento generado en list_cup_dates será la fecha del último cupón recibido del enfoque backward
                #Ahora lo tenemos que ensamblar con el enfoque forward
                time_count = list_cup_dates[0]
                time_count += datetime.timedelta(i_sec_cup_feq*365/12)
                while fut_cup_end_date >= time_count:
                    list_cup_dates += [time_count]
                    time_count += datetime.timedelta(i_sec_cup_feq*365/12)

            except:
                #Si no se ha generado list_cup_dates, se toma la siguiente alternativa
                time_count = i_sec_nxt_cup
                while fut_cup_end_date >= time_count:
                    list_cup_dates += [time_count]
                    time_count += datetime.timedelta(i_sec_cup_feq*365/12)

            if len(list_cup_dates) > 0:            
                i_sec_coupons_df = pd.DataFrame(data={'security': aux_coupons_df.loc[i_idx, 'security'],
                                                      'Coupon_Payment_Dt': list_cup_dates, 'Cupón': i_sec_cup,
                                                      'Divisa': aux_coupons_df.loc[i_idx, 'Divisa'],
                                                      'Banco': aux_coupons_df.loc[i_idx, 'Banco']})
                coupons_df = coupons_df.append(i_sec_coupons_df, ignore_index=True)

        except:
            continue

    coupons_df = coupons_df.drop_duplicates()    

    #Una vez generado el DataFrame de forma similar al los dividendos recibidos, el resto de los cálculos se realizan de forma análoga
    list_amount_cup = []
    for i in range(len(coupons_df)):
        aux_mat = global_mat.loc[global_mat['security'].isin([coupons_df['security'].iloc[i]]) & global_mat['Banco'].isin([coupons_df['Banco'].iloc[i]])][['date', 'Nominal']]
        try:
            date_quantity = aux_mat[aux_mat['date'] <= coupons_df['Coupon_Payment_Dt'].iloc[i]]['Nominal'].tail(1).values[0]
            list_amount_cup += [date_quantity * coupons_df['Cupón'].iloc[i]]        
        except: #Si no hay nada comprado para el activo en el que en esa fecha se paga un dividendo
            list_amount_cup += [0]

    ############################# INCLUIR ALGO SIMILAR A LO DE LAS PREFERENTES EN LOS CUPONES --------------------------------- 

    #Incluir preferentes
    if len(global_mat.loc[global_mat['Clasificación'].isin(['Preferentes'])])>0:

        pref_carry_df = pd.DataFrame(data={})

        pref_list = list(global_mat.loc[global_mat['Clasificación'].isin(['Preferentes'])]['security'].unique())

        for eq in pref_list:
            i_start_date = datetime.datetime.strptime(date_st.value, '%d-%m-%Y')
            i_end_date = datetime.datetime.strptime(date_end.value, '%d-%m-%Y')

            try:
                request = bql.Request(eq, {'Cupón': f.dropna(d.cash_divs(CA_ADJ='RAW', dates = f.range(i_start_date, i_end_date), ca_date_type = 'PAY_DATE', currency = ac_curncy.value))})
                res = bq.execute(request)
                aux_res = res[0].df().reset_index()
                aux_res['aux'] = range(len(aux_res)) #Se Asigna una variable auxiliar para identificar dónde se tienen que colocar las fechas ex-dividend

                #Para buscar las fechas Ex-Dividend, se va a coger el último valor del dividendo pagado y al primero se le resta 1 año.
                #No obstante, en vez de sacar el histórico, sacamos el número mínimo necesario de las fechas ex-dividends que necesitamos con la función last
                #Se alinean los valores con la variable auxiliar creada al hacer un merge

                if len(aux_res) > 1:
                    aux_st_dt = aux_res['DATE'].values[0]
                    aux_end_dt = aux_res['DATE'].values[-1]

                else:
                    aux_st_dt = aux_res['DATE'].values[0]
                    aux_end_dt = aux_res['DATE'].values[0]

                pay_st_dt = pd.to_datetime(str(aux_st_dt - np.timedelta64(8760, 'h'))).strftime('%Y-%m-%d') 
                pay_end_dt = pd.to_datetime(str(aux_end_dt)).strftime('%Y-%m-%d')

                request = bql.Request(eq, {'Cupón': f.last(f.dropna(d.cash_divs(CA_ADJ='RAW', dates = f.range(pay_st_dt, pay_end_dt), ca_date_type = 'EFFECTIVE_DATE', currency = ac_curncy.value)), n = len(aux_res))})
                res = bq.execute(request)
                aux_res2 = res[0].df().reset_index().rename(index=str, columns={'DATE': 'Ex-div_date'})
                aux_res2['aux'] = range(len(aux_res2))

                aux_res = pd.merge(aux_res, aux_res2[['aux', 'Ex-div_date']], how = 'left', on = ['aux'])

                #Aquí se aproximan los pagos de las preferentes estimando la frecuencia de pago -------------------

                request = bql.Request(eq, {'Cupón': f.last(f.dropna(d.cash_divs(dividend_type = 'REGULAR', CA_ADJ='RAW',dates=f.range('-2Y', end_dt), currency = ac_curncy.value)), n=2)})
                res = bq.execute(request)
                aux_pref_fut = res[0].df().reset_index()

                #Se obtiene el valor del último pago, que se supondrá que es el que se obtendrá a futuro
                est_cup_pref = aux_pref_fut['Cupón'][0]    

                #Se aproxima la frecuencia de los pagos
                est_freq_pref = round(((aux_pref_fut['DATE'][1] - aux_pref_fut['DATE'][0]).days)/31)

                #Se calculan las fechas estimadas de los pagos de las preferentes
                list_cup_dates_fut = []
                time_count_pref = aux_res['DATE'].iloc[-1].to_pydatetime() #Se coge la última fecha obtenida según la fecha especificada en date_end.value
                time_count_pref += datetime.timedelta(est_freq_pref*365/12)
                while fut_cup_end_date >= time_count:
                    list_cup_dates_fut += [time_count]
                    time_count_pref += datetime.timedelta(est_freq_pref*365/12)

                #Se une con el df auxiliar generado anterior
                aux_res2 = pd.DataFrame(data={'ID': np.nan, 'DATE': list_cup_dates_fut, 'CURRENCY': np.nan, 'Cupón': est_cup_pref, 'aux': np.nan, 'Ex-div_date': np.nan})
                aux_res = aux_res.append(aux_res2, ignore_index = True).fillna(method='ffill', axis = 0)

                #-----------------------------------------------------------------------------------------------------------

                pref_carry_df = pref_carry_df.append(aux_res, ignore_index = True)

            except:
                pass

        pref_carry_df = pref_carry_df.dropna(subset=['Cupón'])
        pref_carry_df = pref_carry_df.drop(['aux'], axis=1)
        
        df_tick_banks = global_mat[['Banco', 'security']].drop_duplicates()
        df_tick_banks = df_tick_banks.rename(index=str, columns={'security': 'ID'}) 

        pref_carry_df = pd.merge(pref_carry_df, df_tick_banks, how='left', on=['ID']) 
        
        #Se genera una pivot table donde las filas son los activos y las cantidas y las columnas son los periodos temporales que se han cargado
        n_assets = pd.DataFrame(data={})
        for i_bank in global_mat['Banco'].unique():
            aux_i_global_mat = global_mat.loc[global_mat['Banco'].isin([i_bank])]
            n_assets = pd.concat([n_assets, aux_i_global_mat[['date', 'Banco', 'security', 'Nominal']].pivot_table(index = ['Banco', 'security'], columns = 'date', values = 'Nominal').fillna(0).reset_index()], ignore_index=True)
        list_dates_concat = []
        for i_dates_concat in n_assets.columns:
            if i_dates_concat != 'Banco' and i_dates_concat != 'security':
                list_dates_concat += [i_dates_concat]
        n_assets = n_assets[['Banco', 'security'] + sorted(list_dates_concat)]
        n_assets = n_assets.fillna(method='ffill', axis = 1)
        
        
        
        list_amount_pref = []
        for i in range(len(pref_carry_df)):
            aux_mat = n_assets.loc[n_assets['security'].isin([pref_carry_df['ID'].iloc[i]]) & n_assets['Banco'].isin([pref_carry_df['Banco'].iloc[i]])].drop(['Banco', 'security'], axis=1) 

            try:
                if aux_mat.loc[:,aux_mat.columns <= pref_carry_df['Ex-div_date'].iloc[i]].iloc[:,-1].values[0] != 0: #Si no se ha comprado el valor antes de la fecha ex-dividend, entonces no se podrá computar el dividendo recibido en la fecha de su pago            
                    date_quantity = aux_mat.loc[:,aux_mat.columns <= pref_carry_df['DATE'].iloc[i]].iloc[:,-1].values[0]
                    list_amount_pref += [date_quantity * pref_carry_df['Cupón'].iloc[i]]

                else:
                    list_amount_pref += [0]

            except: #Si no hay nada comprado para el activo en el que en esa fecha se paga el carry del preferente
                list_amount_pref += [0]
        
        #Adaptar el formato de los pagos de las preferentes (similar al de dividendos) al de los cupones
        pref_carry_df = pref_carry_df.rename(index=str, columns={'ID': 'security', 'DATE': 'Coupon_Payment_Dt', 'CURRENCY': 'Divisa'})
        
        #Agrupar las preferentes a los cupones        
        coupons_df = coupons_df.append(pref_carry_df[['security', 'Coupon_Payment_Dt', 'Cupón', 'Divisa', 'Banco']], ignore_index=True)    
        
        list_amount_cup = list_amount_cup + list_amount_pref    
    
    ##################################### --------------------------------------------------------------------------
    
    coupons_df['Total_Coupon'] = list_amount_cup        
    coupons_df['mes'] = coupons_df['Coupon_Payment_Dt'].map(lambda x: x.month)

    #En caso de que algún bono esté en otra divisa, calcular el cambio de divisa
    list_real_amount_cup = []
    for i_idx in coupons_df.index:
        divisa_activo = coupons_df.loc[i_idx, 'Divisa']
        if divisa_activo != ac_curncy.value:
            
            #Al calcular las fechas futuras en las que se recibirían los cupones, en aquellos casos en los que se necesita extraer el valor de la 
            #divisa para una fecha futura, se aplicará el valor de la divisa en la fecha fin parametrizada del informe
            if coupons_df.loc[i_idx, 'Coupon_Payment_Dt'].to_pydatetime() < datetime.datetime.strptime(date_end.value, '%d-%m-%Y'):
                fecha_forward = coupons_df.loc[i_idx, 'Coupon_Payment_Dt'].to_pydatetime()
                
            else:
                fecha_forward = datetime.datetime.strptime(date_end.value, '%d-%m-%Y')
            
            request =  bql.Request(ac_curncy.value + divisa_activo + ' Curncy', {'FX': d.px_last(start = fecha_forward, fill = 'prev')['value']})
            res = bq.execute(request)
            fx_df = res[0].df()

            list_real_amount_cup += [coupons_df.loc[i_idx, 'Total_Coupon'] / fx_df['FX'][0]]
        else:
            list_real_amount_cup += [coupons_df.loc[i_idx, 'Total_Coupon']]

    coupons_df['Real_Total_Coupon'] = list_real_amount_cup       

    cum_coupons_df = pd.pivot_table(coupons_df, values='Real_Total_Coupon', index=['security'],columns=['mes'], aggfunc=np.sum)

    def_forward_mat_cup = pd.DataFrame(data={})
    for col in cum_coupons_df.columns.tolist():
        def_forward_mat_cup[str(col)] = cum_coupons_df[col].tolist()

    if cum_coupons_df.columns.tolist() != list(range(1,13)): #Ver si faltan meses por incluir en el año
        for left_month in list(set(list(range(1,13))) - set(cum_coupons_df.columns.tolist())):
            def_forward_mat_cup[str(left_month)] = np.nan

    def_forward_mat_cup['Ticker'] = cum_coupons_df.index.tolist() 
    def_forward_mat_cup = def_forward_mat_cup[['Ticker'] + [str(i) for i in range(1,13)]] #Ordenar la matriz
    def_forward_mat_cup['Total'] = def_forward_mat_cup.sum(axis=1,skipna=True)
    def_forward_mat_cup = def_forward_mat_cup.append(def_forward_mat_cup.sum(numeric_only=True), ignore_index=True)
    def_forward_mat_cup.loc[len(def_forward_mat_cup)-1, 'Ticker'] = 'Total'
    
def gen_excel_type_table (tgt = None):
    global agg_global_data_mat
    global matriz_filtrada1 #Matriz filtrada por casificación sin cambios
    
    output_number_assets_class.value = str(agg_global_data_mat['Clasificación'].isin([assets_class.value]).sum())
    
    try:
        
        if assets_class.value == 'Acciones' or assets_class.value == 'ETFs RV' or assets_class.value == 'Fondos RV':
            matriz_filtrada1 = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin([assets_class.value])]
            #Se seleccionan los campos necesarios para esta tipología, excluyendo aquellos que no pertenecen a la misma
            matriz_filtrada1 = matriz_filtrada1[matriz_filtrada1.columns[~matriz_filtrada1.columns.isin(['ID_BBG', 'ISIN', 'Rating', 'Cupón', 'Vencimiento', 'Fecha Compra', 'P0_Report', 'Dividendos recibidos', 'Intereses recibidos', 'Dividend yield fecha compra', 'Interests'])]]           
            print_excel_type_table_Equity (matriz_filtrada1)
            
        elif assets_class.value == 'Bonos' or assets_class.value == 'ETFs RF' or assets_class.value == 'Fondos RF' or assets_class.value == 'Otros' or assets_class.value == 'Sin Clasificar' or assets_class.value == 'Preferentes' or assets_class.value == 'Tesoros':
            matriz_filtrada1 = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin([assets_class.value])]
            #Se seleccionan los campos necesarios para esta tipología, excluyendo aquellos que no pertenecen a la misma
            matriz_filtrada1 = matriz_filtrada1[matriz_filtrada1.columns[~matriz_filtrada1.columns.isin(['ID_BBG', 'ISIN', 'Vencimiento', 'Fecha Compra', 'P0_Report', 'Dividendos recibidos', 'Intereses recibidos', 'Dividend yield fecha compra', 'Interests'])]]
            print_excel_type_table_FI (matriz_filtrada1)
        
        elif assets_class.value == 'CASH':
            matriz_filtrada1 = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin([assets_class.value])]
            #Se seleccionan los campos necesarios para esta tipología, excluyendo aquellos que no pertenecen a la misma
            matriz_filtrada1 = matriz_filtrada1[['Banco', 'security', 'Título', 'Divisa', 'País']] 
            print_excel_type_table_CASH (matriz_filtrada1)            

    except: #No se parametrizan los Estrucutrados u otras tipologías
        pass       

def print_excel_type_table_Equity (gen_table):
    #Cargar valores en las siguientes variables donde hay widgets
    global tabla_excel    
    
    #Generar tabla con inputs modificables tipo Excel
    tabla_excel.children = () #En caso de que se vuelva a llamar al Dropdown, borrar la anterior cargada
    
    #Generar los valores del Dataframe
    aux_v = []
    for vals_df in gen_table.values:
        aux_h = []
        i = 0
        #Hay que poner la expresión try-except para los NAs que haya
        for val_df in vals_df:
            if i == 0: #Clasificación
                aux_h += [Dropdown(value = val_df, options = ['Acciones', 'Fondos RV', 'ETFs RV', 'Bonos', 'Fondos RF', 'ETFs RF', 'Otros', 'CASH', 'Preferentes', 'Tesoros' ,'Sin Clasificar'], layout=Layout(width='140px'))]
            elif i == 1: #Banco
                try:
                    aux_h += [Text(value = val_df, layout=Layout(width='120px'))]
                except:
                    aux_h += [Text(value = str(val_df), layout=Layout(width='120px'))]
            elif i == 2: #security
                try:
                    aux_h += [Text(value = val_df, disabled = True, layout=Layout(width='135px'))]
                except:
                    aux_h += [Text(value = str(val_df), disabled = True, layout=Layout(width='135px'))]
            elif i == 3: #Título
                try:
                    aux_h += [Text(value = val_df, layout=Layout(width='145px'))]
                except:
                    aux_h += [Text(value = str(val_df), layout=Layout(width='145px'))]
            elif i == 4: #Divisa
                try:
                    aux_h += [Text(value = val_df, layout=Layout(width='65px'))]
                except:
                    aux_h += [Text(value = str(val_df), layout=Layout(width='65px'))]                
            elif i == 5: #País
                try:
                    aux_h += [Text(value = val_df, layout=Layout(width='120px'))]
                except:
                    aux_h += [Text(value = str(val_df), layout=Layout(width='120px'))]
            elif i == 6: #Sector
                try:
                    aux_h += [Text(value = val_df, layout=Layout(width='100px'))]
                except:
                    aux_h += [Text(value = str(val_df), layout=Layout(width='100px'))]                    
            elif i == 7: #Nominal
                aux_h += [Text(value = str(val_df), layout=Layout(width='80px'))]
            elif i == 8: #Precio compra
                aux_h += [Text(value = str(val_df), layout=Layout(width='90px'))]                
            elif i == 9: #Precio compra en divisa reportada
                aux_h += [Text(value = str(val_df), layout=Layout(width='90px'))]                                          
            elif i == 10: #Precio Actual
                aux_h += [Text(value = str(val_df), layout=Layout(width='90px'))]
            elif i == 11: #P1_Report
                aux_h += [Text(value = str(val_df), layout=Layout(width='90px'))]
            i += 1
        aux_v += [HBox(aux_h)]
    
    #Generar las cabeceras del Dataframe como cabeceras del Excel que no se puedan modificar    
    i = 0
    aux_head = []
    for head in gen_table.columns:
        
        if i == 0:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='140px'))]
        elif i == 1:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='120px'))]
        elif i == 2:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='135px'))]
        elif i == 3:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='145px'))]
        elif i == 4:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='65px'))]
        elif i == 5:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='120px'))]
        elif i == 6:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='100px'))]            
        elif i == 7:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='80px'))]
        elif i == 8:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='90px'))]            
        elif i == 9:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='90px'))]
        elif i == 10:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='90px'))]      
        elif i == 11:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='90px'))]                 
        i += 1

    tabla_excel.children += (VBox([HBox(aux_head)] +  aux_v),)

def print_excel_type_table_FI (gen_table):
    #Cargar valores en las siguientes variables donde hay widgets
    global tabla_excel    
    
    #Generar tabla con inputs modificables tipo Excel
    tabla_excel.children = () #En caso de que se vuelva a llamar al Dropdown, borrar la anterior cargada
    
    #Generar los valores del Dataframe
    aux_v = []
    for vals_df in gen_table.values:
        aux_h = []
        i = 0
        #Hay que poner la expresión try-except para los NAs que haya
        for val_df in vals_df:
            if i == 0: #Clasificación
                aux_h += [Dropdown(value = val_df, options = ['Acciones', 'Fondos RV', 'ETFs RV', 'Bonos', 'Fondos RF', 'ETFs RF', 'Otros', 'CASH', 'Preferentes', 'Tesoros', 'Sin Clasificar'], layout=Layout(width='140px'))]
            elif i == 1: #Banco
                try:
                    aux_h += [Text(value = val_df, layout=Layout(width='120px'))]
                except:
                    aux_h += [Text(value = str(val_df), layout=Layout(width='120px'))]
            elif i == 2: #security
                try:
                    aux_h += [Text(value = val_df, disabled = True, layout=Layout(width='135px'))]
                except:
                    aux_h += [Text(value = str(val_df), disabled = True, layout=Layout(width='135px'))]
            elif i == 3: #Título
                try:
                    aux_h += [Text(value = val_df, layout=Layout(width='145px'))]
                except:
                    aux_h += [Text(value = str(val_df), layout=Layout(width='145px'))]
            elif i == 4: #Divisa
                try:
                    aux_h += [Text(value = val_df, layout=Layout(width='65px'))]
                except:
                    aux_h += [Text(value = str(val_df), layout=Layout(width='65px'))]
            elif i == 5: #Rating
                try:
                    aux_h += [Text(value = val_df, layout=Layout(width='65px'))]
                except:
                    aux_h += [Text(value = str(val_df), layout=Layout(width='65px'))]                         
            elif i == 6: #País
                try:
                    aux_h += [Text(value = val_df, layout=Layout(width='120px'))]
                except:
                    aux_h += [Text(value = str(val_df), layout=Layout(width='120px'))]  
            elif i == 7: #Sector
                try:
                    aux_h += [Text(value = val_df, layout=Layout(width='100px'))]
                except:
                    aux_h += [Text(value = str(val_df), layout=Layout(width='100px'))]                      
            elif i == 8: #Nominal
                aux_h += [Text(value = str(val_df), layout=Layout(width='80px'))]
            elif i == 9: #Cupón
                aux_h += [Text(value = str(val_df), layout=Layout(width='55px'))]                     
            elif i == 10: #Precio compra
                aux_h += [Text(value = str(val_df), layout=Layout(width='90px'))]
            elif i == 11: #Precio compra en divisa Reportada
                aux_h += [Text(value = str(val_df), layout=Layout(width='90px'))]                     
            elif i == 12: #Precio Actual
                aux_h += [Text(value = str(val_df), layout=Layout(width='90px'))]
            elif i == 13: #P1_Report
                aux_h += [Text(value = str(val_df), layout=Layout(width='90px'))]                     
            i += 1
        aux_v += [HBox(aux_h)]
    
    #Generar las cabeceras del Dataframe como cabeceras del Excel que no se puedan modificar    
    i = 0
    aux_head = []
    for head in gen_table.columns:
        
        if i == 0:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='140px'))]
        elif i == 1:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='120px'))]
        elif i == 2:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='135px'))]
        elif i == 3:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='145px'))]
        elif i == 4:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='65px'))]
        elif i == 5:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='65px'))]            
        elif i == 6:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='120px'))]
        elif i == 7:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='100px'))]            
        elif i == 8:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='80px'))]
        elif i == 9:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='55px'))]
        elif i == 10:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='90px'))]        
        elif i == 11:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='90px'))]      
        elif i == 12:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='90px'))]      
        elif i == 13:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='90px'))]             
        i += 1

    tabla_excel.children += (VBox([HBox(aux_head)] +  aux_v),)
    
def print_excel_type_table_CASH (gen_table):
    #Cargar valores en las siguientes variables donde hay widgets
    global tabla_excel    
    
    #Generar tabla con inputs modificables tipo Excel
    tabla_excel.children = () #En caso de que se vuelva a llamar al Dropdown, borrar la anterior cargada
    
    #Generar los valores del Dataframe
    aux_v = []
    for vals_df in gen_table.values:
        aux_h = []
        i = 0
        #Hay que poner la expresión try-except para los NAs que haya
        for val_df in vals_df:
            if i == 0: #Banco
                try:
                    aux_h += [Text(value = val_df, layout=Layout(width='120px'))]
                except:
                    aux_h += [Text(value = str(val_df), layout=Layout(width='120px'))]
            elif i == 1: #security
                try:
                    aux_h += [Text(value = val_df, disabled = True, layout=Layout(width='135px'))]
                except:
                    aux_h += [Text(value = str(val_df), disabled = True, layout=Layout(width='135px'))]
            elif i == 2: #Título
                try:
                    aux_h += [Text(value = val_df, layout=Layout(width='145px'))]
                except:
                    aux_h += [Text(value = str(val_df), layout=Layout(width='145px'))]
            elif i == 3: #Divisa
                try:
                    aux_h += [Text(value = val_df, layout=Layout(width='65px'))]
                except:
                    aux_h += [Text(value = str(val_df), layout=Layout(width='65px'))]                
            elif i == 4: #País
                try:
                    aux_h += [Text(value = val_df, layout=Layout(width='120px'))]
                except:
                    aux_h += [Text(value = str(val_df), layout=Layout(width='120px'))]
            i += 1
        aux_v += [HBox(aux_h)]
    
    #Generar las cabeceras del Dataframe como cabeceras del Excel que no se puedan modificar    
    i = 0
    aux_head = []
    for head in gen_table.columns:
        
        if i == 0:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='120px'))]
        elif i == 1:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='135px'))]
        elif i == 2:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='145px'))]
        elif i == 3:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='65px'))]
        elif i == 4:
            aux_head += [Text(value = head, disabled = True, layout=Layout(width='120px'))]                
        i += 1

    tabla_excel.children += (VBox([HBox(aux_head)] +  aux_v),)    
        
def apply_changes_excel_type_table (tgt = None):
    global agg_global_data_mat
    global list_unique_clasi    
    
    request_button6.disabled = True #Durante el proceso de esta descarga de datos, se desabilita el botón para evitar errores de doble click
    
    cur_val_selected = assets_class.value
    
    read_excel_type_table()
    print_message_box.value = 'CAMBIOS APLICADOS EN ' + str(assets_class.value)
    
    #Imprimir de nuevo la tabla 
    list_unique_clasi = list(agg_global_data_mat['Clasificación'].unique())
    assets_class.options = list_unique_clasi    
    
    if cur_val_selected in list_unique_clasi:
        assets_class.value  = cur_val_selected
    else:
        assets_class.value = list_unique_clasi[0]    
        
    request_button6.disabled = False #Durante el proceso de esta descarga de datos, se desabilita el botón para evitar errores de doble click
        
def read_excel_type_table (tgt = None):
    global global_mat
    global agg_global_data_mat
    global matriz_filtrada1 #Matriz filtrada por casificación sin cambios
    global matriz_filtrada2 #Matriz filtrada por casificación CON CAMBIOS DEL USUARIO

    #Definir el Dataframe vacío con las mismas columnas e índices
    matriz_filtrada2 = pd.DataFrame(data={})
    matriz_filtrada2 = matriz_filtrada2.reindex(index = matriz_filtrada1.index, columns = matriz_filtrada1.columns)

    #Leer las celdas, si no son valores numéricos serán carácteres 
    for cells,idx in zip(tabla_excel.children[0].children[1:], matriz_filtrada2.index): #Saltar las cabeceras
        for cell, col in zip(cells.children, matriz_filtrada2.columns):
            try:
                matriz_filtrada2.loc[idx,col] = float(cell.value)
            except:
                matriz_filtrada2.loc[idx,col] = cell.value

    #Identificar si hay diferencias entre matriz_filtrada1 y matriz_filtrada2 
    if matriz_filtrada1.equals(matriz_filtrada2) == False:
        contrast_mat = matriz_filtrada1 == matriz_filtrada2
        
        #Se tiene que hacer una copia para que no salte un warning de que estás machacando valores
        agg_global_data_mat2 = agg_global_data_mat.copy()
        
        for i_contrast in contrast_mat.index:
            for j_contras in contrast_mat.columns:
                if contrast_mat.loc[i_contrast, j_contras] == False:
                    agg_global_data_mat2.loc[i_contrast, j_contras] = matriz_filtrada2.loc[i_contrast, j_contras]
        
        #Se aplican esos cambios en la matriz desglosada, para lo cual se eliminan las filas con los valores antiguos y se añaden los valores modificados
#         agg_global_data_mat = agg_global_data_mat.drop(agg_global_data_mat.index[agg_global_data_mat2.index.tolist()])
#         agg_global_data_mat = agg_global_data_mat.append(agg_global_data_mat2).sort_index()
        agg_global_data_mat = agg_global_data_mat2
        
        #Si ha habido cambios en la Clasificación, se aplican sobre la matriz global
        sample_agg_global_data_mat = agg_global_data_mat[['security', 'Clasificación']].drop_duplicates()

        list_new_clasi = []
        for idx in global_mat.index:
            if global_mat.loc[idx, 'security'] in sample_agg_global_data_mat['security'].tolist():
                list_new_clasi += [sample_agg_global_data_mat.loc[sample_agg_global_data_mat['security'].isin([global_mat.loc[idx, 'security']]), 'Clasificación'].values[0]]
            else:
                list_new_clasi += [global_mat.loc[idx, 'Clasificación']]

        global_mat = global_mat.drop(['Clasificación'], axis=1)
        global_mat['Clasificación'] = list_new_clasi        

def Load_Prices_in_Excel (tgt = None):
    global agg_global_data_mat

    #Se carga el Excel que se utiliza como soporte de precios para volcar los precios reales desde otro EXcel
    #Sobre SOPORTE_Precios_Portfolio se sobreescriben las pestañas con los valores

#     book = load_workbook('SOPORTE_Precios_Portfolio.xlsx')
#     writer = pd.ExcelWriter('SOPORTE_Precios_Portfolio.xlsx', engine='openpyxl') 
#     writer.book = book
#     writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

#     # Añadir una pestaña auxiliar para que al borrarlas al menos en el Excel quede una y no salte un error
#     pestana_blanco = pd.DataFrame(data={})
#     pestana_blanco.to_excel(writer, 'aux_sheet')
#     writer.save()

#     #Borrar pestañas con datos
#     book.remove_sheet(book.get_sheet_by_name('Precios'))
#     writer.save()

#     #Crearlas nuevamente para que estén vacías
#     book.create_sheet('Precios')
#     writer.save()

#     #Borrar la pestaña auxuliar
#     book.remove_sheet(book.get_sheet_by_name('aux_sheet'))
#     writer.save()    

#     #Vuelve a cargar el Excel
#     book = load_workbook('SOPORTE_Precios_Portfolio.xlsx')
#     writer = pd.ExcelWriter('SOPORTE_Precios_Portfolio.xlsx', engine='openpyxl') 
#     writer.book = book
#     writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

#     #Cargar valores nuevos en las pestañas del Excel SOPORTE_Precios_Portfolio
#     agg_global_data_mat[['Banco', 'security', 'ISIN', 'Título', 'Precio compra', 'Precio compra Report', 'Precio Actual', 'P1_Report']].to_excel(writer, 'Precios', index=False)

#     writer.save()
    
def Read_Prices_from_Excel (tgt = None):
    global agg_global_data_mat
    
    read_excel_button.disabled = True #Durante el proceso de esta descarga de datos, se desabilita el botón para evitar errores de doble click
    
#     new_prices_data = pd.read_excel('SOPORTE_Precios_Portfolio.xlsx')
#     list_columns_mat = agg_global_data_mat.columns
    
#     agg_global_data_mat = agg_global_data_mat.drop(['Precio compra', 'Precio compra Report', 'Precio Actual', 'P1_Report'], axis=1)    
#     agg_global_data_mat = pd.merge(agg_global_data_mat, new_prices_data[['Banco', 'security', 'ISIN', 'Precio compra', 'Precio compra Report', 'Precio Actual', 'P1_Report']], how='left', on=['Banco', 'security', 'ISIN'])
#     agg_global_data_mat = agg_global_data_mat[list_columns_mat]    
    
#     print_message_box.value = 'Se han leído los precios de SOPORTE_Precios_Portfolio.xlsx'
    
#     #Imprimir de nuevo la tabla 
#     cur_val_selected = assets_class.value
#     list_unique_clasi = list(agg_global_data_mat['Clasificación'].unique())
#     assets_class.options = list_unique_clasi    
    
#     if cur_val_selected in list_unique_clasi:
#         assets_class.value  = cur_val_selected
#     else:
#         assets_class.value = list_unique_clasi[0]    
        
    read_excel_button.disabled = False #Durante el proceso de esta descarga de datos, se desabilita el botón para evitar errores de doble click        
        
def validate_data (tgt = None):
    global agg_global_data_mat
    global def_mat_div
    global def_mat_cup
    
    if agg_global_data_mat['Clasificación'].isin(['Sin Clasificar']).sum() == 0: 
        
        try:
            def_mat_div = pd.merge(def_mat_div, agg_global_data_mat[['security', 'Clasificación']].drop_duplicates(keep='first'), how='left', on=['security']) 

            #Hacer una reclasificación entre la matriz de dividendos y la de cupones para que los dividendos de los ETF's RF y Fondos RF salgan en la matriz de cupones
            if len(def_mat_div.loc[def_mat_div['Clasificación'].isin(['ETFs RF', 'Fondos RF'])]) > 0:

                def_mat_cup = def_mat_cup.append(def_mat_div.loc[def_mat_div['Clasificación'].isin(['ETFs RF', 'Fondos RF'])], ignore_index=True)
                def_mat_cup = def_mat_cup.drop(['Clasificación'], axis=1)
                def_mat_cup = def_mat_cup.rename(index=str, columns={'security': 'Ticker'})

            def_mat_div = def_mat_div.loc[~def_mat_div['Clasificación'].isin(['ETFs RF', 'Fondos RF'])]
            def_mat_div = def_mat_div.drop(['Clasificación'], axis=1)
            def_mat_div = def_mat_div.rename(index=str, columns={'security': 'Ticker'})     

            def_mat_div = def_mat_div[['Ticker'] + [str(i) for i in range(1,13)]] #Ordenar la matriz
            def_mat_div['Total'] = def_mat_div.sum(axis=1,skipna=True)
            def_mat_div = def_mat_div.append(def_mat_div.sum(numeric_only=True), ignore_index=True)
            def_mat_div.loc[len(def_mat_div)-1, 'Ticker'] = 'Total'
            def_mat_div = def_mat_div.loc[def_mat_div['Total'] != 0]
            
        except:
            pass
        
        try:
            #Aplicar los formatos del GR
            if 'security' in def_mat_cup.columns:
                def_mat_cup = def_mat_cup.rename(index=str, columns={'security': 'Ticker'})            
            
            def_mat_cup = def_mat_cup[['Ticker'] + [str(i) for i in range(1,13)]] #Ordenar la matriz
            def_mat_cup['Total'] = def_mat_cup.sum(axis=1,skipna=True)
            def_mat_cup = def_mat_cup.append(def_mat_cup.sum(numeric_only=True), ignore_index=True)
            def_mat_cup.loc[len(def_mat_cup)-1, 'Ticker'] = 'Total'
            def_mat_cup = def_mat_cup.loc[def_mat_cup['Total'] != 0]
        
        except:
            pass


        #Generar el resto de campos necesarios en agg_global_data_mat relativos a los cálculos de valor actual y retorno
        agg_global_data_mat['Valor de Compra'] = agg_global_data_mat['Precio compra'] * agg_global_data_mat['Nominal']
        
        #Calculamos correctamente el dividend yield, ajustado por los nuevos precios de compra introducidos en Load/Read prices:
        agg_global_data_mat['Dividend yield fecha compra'] = agg_global_data_mat['Dividend yield fecha compra']/agg_global_data_mat['Precio compra Report']

        #Dividir entre 100 el precio de los bonos para obtener el valor de compra
        aux_list_precio_compra = []
        for i in agg_global_data_mat.index:
            if (agg_global_data_mat.loc[i, 'Clasificación'] == 'Bonos') or (agg_global_data_mat.loc[i, 'Clasificación'] == 'Tesoros'):
                aux_list_precio_compra += [agg_global_data_mat.loc[i, 'Precio compra']/100]
            else:
                aux_list_precio_compra += [agg_global_data_mat.loc[i, 'Precio compra']]

        agg_global_data_mat['aux_Precio compra'] = aux_list_precio_compra
        agg_global_data_mat['Valor de Compra'] = agg_global_data_mat['aux_Precio compra'] * agg_global_data_mat['Nominal']
        agg_global_data_mat = agg_global_data_mat.drop(['aux_Precio compra'], axis=1)        

        agg_global_data_mat['Dividendos+Intereses'] = agg_global_data_mat['Dividendos recibidos'].fillna(0) + agg_global_data_mat['Intereses recibidos'].fillna(0)

        #Dividir entre 100 el precio de los bonos para obtener el valor de compra
        aux_list_p1_report = []
        aux_list_p0_report = []
        for i in agg_global_data_mat.index:
            if (agg_global_data_mat.loc[i, 'Clasificación'] == 'Bonos') or (agg_global_data_mat.loc[i, 'Clasificación'] == 'Tesoros'):
                aux_list_p1_report += [agg_global_data_mat.loc[i, 'P1_Report']/100]
                aux_list_p0_report += [agg_global_data_mat.loc[i, 'Precio compra Report']/100]

            else:
                aux_list_p1_report += [agg_global_data_mat.loc[i, 'P1_Report']]    
                aux_list_p0_report += [agg_global_data_mat.loc[i, 'Precio compra Report']]

        agg_global_data_mat['aux_P1_Report'] = aux_list_p1_report
        agg_global_data_mat['aux_Precio compra Report'] = aux_list_p0_report

        agg_global_data_mat['Valor Actual'] = agg_global_data_mat['aux_P1_Report'] * agg_global_data_mat['Nominal'] + (agg_global_data_mat['Interests'].fillna(0) * (agg_global_data_mat['P1_Report']/agg_global_data_mat['Precio Actual']).fillna(1)) * agg_global_data_mat['Nominal'] #Se aplica el cambio de divisa al cupón corrido
        agg_global_data_mat = agg_global_data_mat.drop(['Interests', 'ISIN'], axis=1)
        agg_global_data_mat['Retorno Obtenido'] = ((agg_global_data_mat['Valor Actual'] + agg_global_data_mat['Dividendos+Intereses']) / (agg_global_data_mat['aux_Precio compra Report'] * agg_global_data_mat['Nominal'])) - 1
        agg_global_data_mat = agg_global_data_mat.drop(['aux_P1_Report', 'aux_Precio compra Report'], axis=1)
        
        #Parche para imputar correctamente los bonos mexicanos
        if len(agg_global_data_mat.loc[agg_global_data_mat['Divisa'].isin(['MXN']) & agg_global_data_mat['Clasificación'].isin(['Bonos','Preferentes','Tesoros'])]) > 0:
            index_mxn = agg_global_data_mat.loc[agg_global_data_mat['Divisa'].isin(['MXN']) & agg_global_data_mat['Clasificación'].isin(['Bonos','Preferentes','Tesoros'])].index

            agg_global_data_mat2 = agg_global_data_mat.copy()
            for i_idx_mxn in index_mxn:
                agg_global_data_mat2.loc[i_idx_mxn, 'Valor Actual'] = agg_global_data_mat.loc[i_idx_mxn, 'Valor Actual']*100
                agg_global_data_mat2.loc[i_idx_mxn, 'Intereses recibidos'] = agg_global_data_mat.loc[i_idx_mxn, 'Intereses recibidos']*100
                agg_global_data_mat2.loc[i_idx_mxn, 'Dividendos+Intereses'] = np.nansum([agg_global_data_mat.loc[i_idx_mxn, 'Intereses recibidos']*100,agg_global_data_mat.loc[i_idx_mxn, 'Dividendos recibidos']])
            agg_global_data_mat = agg_global_data_mat2
        ######################################################

        agg_global_data_mat['% Total Cartera'] = agg_global_data_mat['Valor Actual'] / sum(agg_global_data_mat['Valor Actual'])
        agg_global_data_mat['% Total RV'] = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Acciones', 'Fondos RV', 'ETFs RV'])]['Valor Actual']/sum(agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Acciones', 'Fondos RV', 'ETFs RV'])]['Valor Actual'])
        agg_global_data_mat['% Total RF'] = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Bonos', 'Fondos RF', 'ETFs RF','Preferentes'])]['Valor Actual']/sum(agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Bonos', 'Fondos RF', 'ETFs RF','Preferentes'])]['Valor Actual'])                    
        
        agg_global_data_mat = agg_global_data_mat.drop_duplicates().reset_index()
        agg_global_data_mat = agg_global_data_mat.drop(['index'], axis=1)
        
        hidden_widgets3.layout.visibility = 'visible'

    else:
        print_message_box.value = 'HAY ACTIVOS SIN CLASIFICAR CORRECTAMENTE'
    
def Global_Reporting_Generation (tgt = None):
    global global_mat
    global agg_global_data_mat
    global def_mat_div
    global def_mat_cup
    global def_forward_mat_cup        
    global df_bancos_manejada
    global dep_ret_mat
    global tot_div
    global tot_cup
    global df_rf_dist_activ

    #Se declaran los dfs necesarios para generar el Global Reporting
    df_liquidez = pd.DataFrame(data={})
    
    df_bonos = pd.DataFrame(data={})
    df_preferentes = pd.DataFrame(data={})
    df_fi_rf = pd.DataFrame(data={})
    df_etf_rf = pd.DataFrame(data={})
    
    df_acciones = pd.DataFrame(data={})
    df_fi_rv = pd.DataFrame(data={})
    df_etf_rv = pd.DataFrame(data={})
    
    df_rf_dist_activ = pd.DataFrame(data={})
    df_rf_dist_pais = pd.DataFrame(data={})
    df_rf_dist_sector = pd.DataFrame(data={})
    
    df_rv_dist_activ = pd.DataFrame(data={})
    df_rv_dist_pais = pd.DataFrame(data={})
    df_rv_dist_sector = pd.DataFrame(data={}) 
    
    if len(agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['CASH','Tesoros'])]) > 0:
        liquidity_per_bank = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Tesoros','CASH'])]
        
        # Fill DataFrame
        for i_bank in liquidity_per_bank['Banco'].unique():
            # Value
            aux_liquidity_per_bank = liquidity_per_bank.loc[liquidity_per_bank['Banco'].isin([i_bank])][['Clasificación','Valor Actual']]
            aux_tesoros = aux_liquidity_per_bank[aux_liquidity_per_bank['Clasificación'] == 'Tesoros'][['Valor Actual']].sum()
            aux_cash = aux_liquidity_per_bank[aux_liquidity_per_bank['Clasificación'] == 'CASH'][['Valor Actual']].sum()

            # Add to the DataFrame
            df_liquidez.loc['CASH',i_bank] = aux_cash[0]
            df_liquidez.loc['Tesoros',i_bank] = aux_tesoros[0]
        
        # Adjust format to excel
        df_liquidez = df_liquidez.reset_index().rename(columns={"index": ""})
        
    
    if len(agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Bonos'])]) > 0:
        df_bonos = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Bonos'])][['Banco', 'Título', 'Divisa', 'País', 'Cupón', 'Vencimiento',
                                                                                                        'Nominal', 'Fecha Compra', 'Precio compra',
                                                                                                        'Valor de Compra', 'Precio Actual',
                                                                                                        'Dividendos+Intereses', 'Valor Actual',
                                                                                                        'Retorno Obtenido', '% Total RF',
                                                                                                        '% Total Cartera']]
        
        df_bonos = df_bonos.rename(columns = {'Dividendos+Intereses': 'Intereses recibidos'})
        df_bonos = df_bonos.sort_values(by=['% Total Cartera'], ascending=False)
        
    if len(agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Preferentes'])]) > 0:
        df_preferentes = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Preferentes'])][['Banco', 'Título', 'Divisa', 'País', 'Cupón', 'Vencimiento',
                                                                                                        'Nominal', 'Fecha Compra', 'Precio compra',
                                                                                                        'Valor de Compra', 'Precio Actual',
                                                                                                        'Dividendos+Intereses', 'Valor Actual',
                                                                                                        'Retorno Obtenido', '% Total RF',
                                                                                                        '% Total Cartera']]
        
        df_preferentes = df_preferentes.rename(columns = {'Dividendos+Intereses': 'Intereses recibidos'})
        df_preferentes = df_preferentes.sort_values(by=['% Total Cartera'], ascending=False)    
    
    if len(agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Fondos RF'])]) > 0:
        df_fi_rf = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Fondos RF'])][['Banco', 'Título', 'Divisa', 'País',
                                                                                                        'Nominal', 'Fecha Compra', 'Precio compra',
                                                                                                        'Valor de Compra', 'Precio Actual',
                                                                                                        'Dividendos+Intereses', 'Valor Actual',
                                                                                                        'Retorno Obtenido', '% Total RF',
                                                                                                        '% Total Cartera']]
        
        df_fi_rf = df_fi_rf.rename(columns = {'Dividendos+Intereses': 'Intereses recibidos'})
        df_fi_rf = df_fi_rf.sort_values(by=['% Total Cartera'], ascending=False)
        
    if len(agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['ETFs RF'])]) > 0:
        df_etf_rf = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['ETFs RF'])][['Banco', 'Título', 'Divisa', 'País',
                                                                                                        'Nominal', 'Fecha Compra', 'Precio compra',
                                                                                                        'Valor de Compra', 'Precio Actual',
                                                                                                        'Dividendos+Intereses', 'Valor Actual',
                                                                                                        'Retorno Obtenido', '% Total RF',
                                                                                                        '% Total Cartera']]
        
        df_etf_rf = df_etf_rf.rename(columns = {'Dividendos+Intereses': 'Intereses recibidos'})
        df_etf_rf = df_etf_rf.sort_values(by=['% Total Cartera'], ascending=False)
    
    if len(agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Acciones'])]) > 0:
        df_acciones = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Acciones'])][['Banco', 'Título', 'Divisa', 'País',
                                                                                                        'Nominal', 'Fecha Compra', 'Precio compra',
                                                                                                        'Valor de Compra', 'Precio Actual',
                                                                                                        'Dividendos+Intereses', 'Valor Actual',
                                                                                                        'Retorno Obtenido', '% Total RV',
                                                                                                        '% Total Cartera']]
        
        df_acciones = df_acciones.rename(columns = {'Dividendos+Intereses': 'Dividendos recibidos'})
        df_acciones = df_acciones.sort_values(by=['% Total Cartera'], ascending=False)
    
    if len(agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Fondos RV'])]) > 0:
        df_fi_rv = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Fondos RV'])][['Banco', 'Título', 'Divisa', 'País',
                                                                                                        'Nominal', 'Fecha Compra', 'Precio compra',
                                                                                                        'Valor de Compra', 'Precio Actual',
                                                                                                        'Dividendos+Intereses', 'Valor Actual',
                                                                                                        'Retorno Obtenido', '% Total RV',
                                                                                                        '% Total Cartera']]
        
        df_fi_rv = df_fi_rv.rename(columns = {'Dividendos+Intereses': 'Dividendos recibidos'})
        df_fi_rv = df_fi_rv.sort_values(by=['% Total Cartera'], ascending=False)

    if len(agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['ETFs RV'])]) > 0:
        df_etf_rv = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['ETFs RV'])][['Banco', 'Título', 'Divisa', 'País',
                                                                                                        'Nominal', 'Fecha Compra', 'Precio compra',
                                                                                                        'Valor de Compra', 'Precio Actual',
                                                                                                        'Dividendos+Intereses', 'Valor Actual',
                                                                                                        'Retorno Obtenido', '% Total RV',
                                                                                                        '% Total Cartera']]
        
        df_etf_rv = df_etf_rv.rename(columns = {'Dividendos+Intereses': 'Dividendos recibidos'})
        df_etf_rv = df_etf_rv.sort_values(by=['% Total Cartera'], ascending=False)
    
    df_all_RF = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Bonos', 'Fondos RF', 'ETFs RF','Preferentes'])]
    
    if len(df_all_RF) > 0:
        df_rf_dist_activ = pd.DataFrame({'DISTRIBUCIÓN POR TIPO DE ACTIVO':df_all_RF.groupby(['Clasificación'])['Valor Actual'].sum()}).reset_index()
        df_rf_dist_activ['DISTRIBUCIÓN POR TIPO DE ACTIVO'] = df_rf_dist_activ['DISTRIBUCIÓN POR TIPO DE ACTIVO']/sum(df_rf_dist_activ['DISTRIBUCIÓN POR TIPO DE ACTIVO'])

        df_rf_dist_pais = pd.DataFrame({'DISTRIBUCIÓN POR PAÍS':df_all_RF.groupby(['País'])['Valor Actual'].sum()}).reset_index()
        df_rf_dist_pais['DISTRIBUCIÓN POR PAÍS'] = df_rf_dist_pais['DISTRIBUCIÓN POR PAÍS']/sum(df_rf_dist_pais['DISTRIBUCIÓN POR PAÍS'])

        df_rf_dist_sector = pd.DataFrame({'DISTRIBUCIÓN POR SECTOR':df_all_RF.groupby(['Sector'])['Valor Actual'].sum()}).reset_index()
        df_rf_dist_sector['DISTRIBUCIÓN POR SECTOR'] = df_rf_dist_sector['DISTRIBUCIÓN POR SECTOR']/sum(df_rf_dist_sector['DISTRIBUCIÓN POR SECTOR'])

    df_all_RV = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Acciones', 'Fondos RV', 'ETFs RV'])]

    if len(df_all_RV) > 0:
        df_rv_dist_activ = pd.DataFrame({'DISTRIBUCIÓN POR TIPO DE ACTIVO':df_all_RV.groupby(['Clasificación'])['Valor Actual'].sum()}).reset_index()
        df_rv_dist_activ['DISTRIBUCIÓN POR TIPO DE ACTIVO'] = df_rv_dist_activ['DISTRIBUCIÓN POR TIPO DE ACTIVO']/sum(df_rv_dist_activ['DISTRIBUCIÓN POR TIPO DE ACTIVO'])

        df_rv_dist_pais = pd.DataFrame({'DISTRIBUCIÓN POR PAÍS':df_all_RV.groupby(['País'])['Valor Actual'].sum()}).reset_index()
        df_rv_dist_pais['DISTRIBUCIÓN POR PAÍS'] = df_rv_dist_pais['DISTRIBUCIÓN POR PAÍS']/sum(df_rv_dist_pais['DISTRIBUCIÓN POR PAÍS'])

        df_rv_dist_sector = pd.DataFrame({'DISTRIBUCIÓN POR SECTOR':df_all_RV.groupby(['Sector'])['Valor Actual'].sum()}).reset_index()
        df_rv_dist_sector['DISTRIBUCIÓN POR SECTOR'] = df_rv_dist_sector['DISTRIBUCIÓN POR SECTOR']/sum(df_rv_dist_sector['DISTRIBUCIÓN POR SECTOR'])

    dist_global_pais = pd.DataFrame({'DISTRIBUCIÓN POR PAÍS':agg_global_data_mat.groupby(['País'])['Valor Actual'].sum()}).reset_index()
    dist_global_pais = dist_global_pais.rename(index=str, columns={'País': 'DISTRIBUCIÓN POR PAÍS', 'DISTRIBUCIÓN POR PAÍS': 'Valor'}) 
    dist_global_pais = dist_global_pais.sort_values(by=['Valor'], ascending=False)
    dist_global_pais['%'] = dist_global_pais['Valor']/sum(dist_global_pais['Valor'])
    dist_global_pais = dist_global_pais.head(15)

    dist_global_sector = pd.DataFrame({'DISTRIBUCIÓN POR SECTOR':agg_global_data_mat.groupby(['Sector'])['Valor Actual'].sum()}).reset_index()
    dist_global_sector = dist_global_sector.rename(index=str, columns={'Sector': 'DISTRIBUCIÓN POR SECTOR', 'DISTRIBUCIÓN POR SECTOR': 'Valor'}) 
    dist_global_sector = dist_global_sector.sort_values(by=['Valor'], ascending=False)
    dist_global_sector['%'] = dist_global_sector['Valor']/sum(dist_global_sector['Valor'])
    dist_global_sector = dist_global_sector.head(15)    
    
    df_divisas = agg_global_data_mat.copy()
    for i in range(len(df_divisas)):
        if df_divisas.loc[i,'Divisa'] == 'USD' or df_divisas.loc[i,'Divisa'] == 'EUR' or df_divisas.loc[i,'Divisa'] == 'MXN' or df_divisas.loc[i,'Divisa'] == 'GBP':
            pass
        else:
            df_divisas.loc[i,'Divisa'] = 'OTRAS'        
            
    #Se obtienen los valores iniciales de la cartera
    aux_df_divisas_0 = global_mat[global_mat['date'].isin([global_mat['date'].unique()[0]])][['Divisa', 'value']]
    for i in aux_df_divisas_0.index:
        if aux_df_divisas_0.loc[i,'Divisa'] == 'USD' or aux_df_divisas_0.loc[i,'Divisa'] == 'EUR' or aux_df_divisas_0.loc[i,'Divisa'] == 'MXN' or aux_df_divisas_0.loc[i,'Divisa'] == 'GBP':
            pass
        else:
            aux_df_divisas_0.loc[i,'Divisa'] = 'OTRAS'           
            
    aux_df_divisas_0 = aux_df_divisas_0.rename(index=str, columns={'value': 'Valor de Compra Report'})
    
    df_divisas = df_divisas.append(aux_df_divisas_0, ignore_index=True)

    df_divisas = df_divisas.groupby(['Divisa'])[['Valor de Compra Report', 'Valor Actual']].sum().reset_index()
    df_divisas = df_divisas.rename(index=str, columns={'Divisa': 'DISTRIBUCÓN POR DIVISA', 'Valor de Compra Report': 'Fecha Inicial', 'Valor Actual': 'Fecha Final'})
    df_divisas['Variación'] = (df_divisas['Fecha Final'] / df_divisas['Fecha Inicial']) - 1   
    df_divisas['% del total'] = df_divisas['Fecha Final'] / sum(df_divisas['Fecha Final'].fillna(0))
    df_divisas = df_divisas[['DISTRIBUCÓN POR DIVISA', 'Fecha Inicial', 'Variación', 'Fecha Final', '% del total']]
    df_divisas = df_divisas.append(pd.DataFrame(data={'DISTRIBUCÓN POR DIVISA':'',
                                                      'Fecha Inicial': df_divisas['Fecha Inicial'].sum(),
                                                      'Variación': '',
                                                      'Fecha Final': df_divisas['Fecha Final'].sum(),
                                                      '% del total': ''}, index=[0]), ignore_index=True)

    df_agg_asset_type = pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': [], 'Fecha Inicial': [], 'Fecha Final': []})
    aux_agg_asset_type_rf = pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': [], 'Fecha Inicial': [], 'Fecha Final': []})
    aux_agg_asset_type_rv = pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': [], 'Fecha Inicial': [], 'Fecha Final': []})
    
    aux_agg_global_data_mat = agg_global_data_mat.copy()
    
    aux_agg_global_data_mat_0 = global_mat[global_mat['date'].isin([global_mat['date'].unique()[0]])][['Clasificación', 'Banco', 'value']]
    aux_agg_global_data_mat_0 = aux_agg_global_data_mat_0.rename(index=str, columns={'value': 'Valor de Compra Report'})
    
    aux_agg_global_data_mat = aux_agg_global_data_mat.append(aux_agg_global_data_mat_0, ignore_index=True)

    agg_asset_type_liq = pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': "1 - Liquidez",
                                            'Fecha Inicial': aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['CASH','Tesoros'])]['Valor de Compra Report'].sum(),
                                            'Fecha Final': aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['CASH','Tesoros'])]['Valor Actual'].sum()}, index=[0])    

    try:
        aux_data = aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['Bonos'])].groupby(['Clasificación'])[['Valor de Compra Report', 'Valor Actual']].sum().reset_index().values[0].tolist()
        aux_agg_asset_type_rf = aux_agg_asset_type_rf.append(pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': '2.1 - Directo: Bonos', 'Fecha Inicial': aux_data[1], 'Fecha Final': aux_data[2]}, index=[0]))
    except:
        aux_agg_asset_type_rf = aux_agg_asset_type_rf.append(pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': '2.1 - Directo: Bonos', 'Fecha Inicial': 0, 'Fecha Final': 0}, index=[0]))

    try:
        aux_data = aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['Fondos RF'])].groupby(['Clasificación'])[['Valor de Compra Report', 'Valor Actual']].sum().reset_index().values[0].tolist()
        aux_agg_asset_type_rf = aux_agg_asset_type_rf.append(pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': '2.2 - Fondos RF', 'Fecha Inicial': aux_data[1], 'Fecha Final': aux_data[2]}, index=[1]))
    except:
        aux_agg_asset_type_rf = aux_agg_asset_type_rf.append(pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': '2.2 - Fondos RF', 'Fecha Inicial': 0, 'Fecha Final': 0}, index=[1]))

    try:
        aux_data = aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['ETFs RF'])].groupby(['Clasificación'])[['Valor de Compra Report', 'Valor Actual']].sum().reset_index().values[0].tolist()
        aux_agg_asset_type_rf = aux_agg_asset_type_rf.append(pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': "2.3 - ETF's", 'Fecha Inicial': aux_data[1], 'Fecha Final': aux_data[2]}, index=[2]))
    except:
        aux_agg_asset_type_rf = aux_agg_asset_type_rf.append(pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': "2.3 - ETF's", 'Fecha Inicial': 0, 'Fecha Final': 0}, index=[2]))    

        
    try:
        aux_data = aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['Preferentes'])].groupby(['Clasificación'])[['Valor de Compra Report', 'Valor Actual']].sum().reset_index().values[0].tolist()
        aux_agg_asset_type_rf = aux_agg_asset_type_rf.append(pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': "2.4 - Preferentes", 'Fecha Inicial': aux_data[1], 'Fecha Final': aux_data[2]}, index=[3]))
    except:
        aux_agg_asset_type_rf = aux_agg_asset_type_rf.append(pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': "2.4 - Preferentes", 'Fecha Inicial': 0, 'Fecha Final': 0}, index=[3]))  
        
        
        
    agg_asset_type_rf = pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': "2 - Renta Fija", 'Fecha Inicial': aux_agg_asset_type_rf['Fecha Inicial'].sum(), 'Fecha Final': aux_agg_asset_type_rf['Fecha Final'].sum()}, index=[0])    

    try:
        aux_data = aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['Acciones'])].groupby(['Clasificación'])[['Valor de Compra Report', 'Valor Actual']].sum().reset_index().values[0].tolist()
        aux_agg_asset_type_rv = aux_agg_asset_type_rv.append(pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': '3.1 - Directo: Acciones', 'Fecha Inicial': aux_data[1], 'Fecha Final': aux_data[2]}, index=[0]))
    except:
        aux_agg_asset_type_rv = aux_agg_asset_type_rv.append(pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': '3.1 - Directo: Acciones', 'Fecha Inicial': 0, 'Fecha Final': 0}, index=[0]))

    try:
        aux_data = aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['Fondos RV'])].groupby(['Clasificación'])[['Valor de Compra Report', 'Valor Actual']].sum().reset_index().values[0].tolist()
        aux_agg_asset_type_rv = aux_agg_asset_type_rv.append(pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': '3.2 - Fondos RV', 'Fecha Inicial': aux_data[1], 'Fecha Final': aux_data[2]}, index=[1]))
    except:
        aux_agg_asset_type_rv = aux_agg_asset_type_rv.append(pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': '3.2 - Fondos RV', 'Fecha Inicial': 0, 'Fecha Final': 0}, index=[1]))

    try:
        aux_data = aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['ETFs RV'])].groupby(['Clasificación'])[['Valor de Compra Report', 'Valor Actual']].sum().reset_index().values[0].tolist()
        aux_agg_asset_type_rv = aux_agg_asset_type_rv.append(pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': "3.3 - ETF's", 'Fecha Inicial': aux_data[1], 'Fecha Final': aux_data[2]}, index=[2]))
    except:
        aux_agg_asset_type_rv = aux_agg_asset_type_rv.append(pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': "3.3 - ETF's", 'Fecha Inicial': 0, 'Fecha Final': 0}, index=[2]))        

    agg_asset_type_rv = pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO': "3 - Renta Variable", 'Fecha Inicial': aux_agg_asset_type_rv['Fecha Inicial'].sum(), 'Fecha Final': aux_agg_asset_type_rv['Fecha Final'].sum()}, index=[0])    

    df_agg_asset_type = df_agg_asset_type.append([agg_asset_type_liq, agg_asset_type_rf,aux_agg_asset_type_rf,
                                                  agg_asset_type_rv, aux_agg_asset_type_rv], ignore_index=True)

    list_weights = []
    for i in range(len(df_agg_asset_type)):
        if i == 0 or i == 1 or i == 6:
            list_weights += [df_agg_asset_type.loc[i, 'Fecha Final']/df_agg_asset_type.loc[[0,1,6], 'Fecha Final'].sum()]
        elif i == 2 or i == 3 or i == 4 or i == 5:
            list_weights += [df_agg_asset_type.loc[i, 'Fecha Final']/df_agg_asset_type.loc[[2,3,4,5], 'Fecha Final'].sum()]
        else:
            list_weights += [df_agg_asset_type.loc[i, 'Fecha Final']/df_agg_asset_type.loc[[7,8,9], 'Fecha Final'].sum()]

    df_agg_asset_type['% del total'] = list_weights
    df_agg_asset_type['Variación'] = ((df_agg_asset_type['Fecha Final'] / df_agg_asset_type['Fecha Inicial']) - 1).fillna(0)   
    df_agg_asset_type = df_agg_asset_type[['DISTRIBUCIÓN POR ACTIVO FINANCIERO', 'Fecha Inicial', 'Variación', 'Fecha Final', '% del total']]
    df_agg_asset_type = df_agg_asset_type.append(pd.DataFrame(data={'DISTRIBUCIÓN POR ACTIVO FINANCIERO':'', 'Fecha Inicial': df_agg_asset_type.loc[[0,1,6],'Fecha Inicial'].sum(),
                                                                    'Variación': '', 'Fecha Final': df_agg_asset_type.loc[[0,1,6],'Fecha Final'].sum(), '% del total': ''}, index=[0]), ignore_index=True)

    
    df_bancos = aux_agg_global_data_mat.groupby(['Banco'])[['Valor de Compra Report', 'Valor Actual']].sum().reset_index()
    df_bancos = df_bancos.rename(index=str, columns={'Valor de Compra Report': 'Fecha Inicial', 'Valor Actual': 'Fecha Final'})
    df_bancos['Tipo'] = 'Custodia'

    #Comprobar si se ha añadido alguna cartera manejada
    try:
        df_bancos_manejada.columns
        df_bancos = df_bancos.append(df_bancos_manejada, ignore_index=True)

    except:
        pass

    agg_bancos = df_bancos.groupby(['Banco'])[['Fecha Inicial', 'Fecha Final']].sum().reset_index()
    agg_bancos['Variación'] = ((agg_bancos['Fecha Final'] / agg_bancos['Fecha Inicial']) - 1).fillna(0)
    agg_bancos['%del total'] = agg_bancos['Fecha Final'] / agg_bancos['Fecha Final'].sum()
    agg_bancos = agg_bancos.append(pd.DataFrame(data={'Banco':'', 'Fecha Inicial': agg_bancos['Fecha Inicial'].sum(), 'Fecha Final': agg_bancos['Fecha Final'].sum(), 'Variación': '', '%del total': ''}, index=[0]), ignore_index=True)

    df_bancos['Variación'] = ((df_bancos['Fecha Final'] / df_bancos['Fecha Inicial']) - 1).fillna(0)

    def_agg_bancos = pd.DataFrame(columns = agg_bancos.columns.tolist())

    for i in range(len(agg_bancos)):
        dict_df = dict(zip(agg_bancos.columns.tolist(), agg_bancos.loc[i].values.tolist()))
        def_agg_bancos = def_agg_bancos.append(pd.DataFrame(data=dict_df, index=[0]), ignore_index=True)
        if i < (len(agg_bancos)-1):
            aux_sub_df_bancos = df_bancos.loc[df_bancos['Banco'].isin([agg_bancos.loc[i].values.tolist()[0]])].sort_values(by='Tipo', ascending=True)
            for idx in aux_sub_df_bancos.index:
                sub_list = aux_sub_df_bancos.loc[idx].values.tolist()
                aux_dict_df = dict(zip(agg_bancos.columns.tolist(), [sub_list[3], sub_list[1], sub_list[2], sub_list[4], '']))
                def_agg_bancos = def_agg_bancos.append(pd.DataFrame(data=aux_dict_df, index=[0]), ignore_index=True)

    def_agg_bancos = def_agg_bancos[['Banco', 'Fecha Inicial', 'Variación', 'Fecha Final', '%del total']]
    def_agg_bancos = def_agg_bancos.rename(index=str, columns={'Banco': 'DISTRIBUCÓN POR BANCO'})

    def_mat_resumen = pd.DataFrame(columns=['Resumen', 'Valor'])
    
#     tot_div = agg_global_data_mat.loc[~agg_global_data_mat['Clasificación'].isin(['Bonos'])]['Dividendos+Intereses'].sum()
#     tot_cup = agg_global_data_mat.loc[agg_global_data_mat['Clasificación'].isin(['Bonos'])]['Dividendos+Intereses'].sum()
    
    try:
        var_exist_test = tot_div - 1
    except:
        tot_div = 0
        
    try:
        var_exist_test = tot_cup - 1
    except:
        tot_cup = 0        
    
    tot_rentas = tot_div + tot_cup 
    
    def_mat_resumen.loc[0] = ['Total activos incio periodo', sum(global_mat[global_mat['date'].isin([global_mat['date'].unique()[0]])]['value'])]
    def_mat_resumen.loc[7] = ['Total activos al final del  periodo', agg_bancos.loc[len(agg_bancos)-1,'Fecha Final']]
    def_mat_resumen.loc[3] = ['Rentas del periodo', tot_rentas]
    def_mat_resumen.loc[4] = ['Dividendos', tot_div]
    def_mat_resumen.loc[5] = ['Cupones', tot_cup]
    def_mat_resumen.loc[2] = ['Cambio en el valor de los activos', def_mat_resumen.loc[7, 'Valor'] - def_mat_resumen.loc[0, 'Valor']]
    def_mat_resumen.loc[6] = ['Variación con intereses y dividendos', def_mat_resumen.loc[3, 'Valor'] + def_mat_resumen.loc[2, 'Valor']]

    #Comprobar si se han añadido depositos y retiros
    try:
        dep_ret_mat.columns
        def_mat_resumen.loc[1] = ['Depósitos y retiros', dep_ret_mat['Valor'].sum()]

    except:
        def_mat_resumen.loc[1] = ['Depósitos y retiros', 0]

    def_mat_resumen = def_mat_resumen.sort_index(axis=0)  
    
    #Se carga el Excel que se utiliza como soporte de datos para generar las gráficas en PowerPoint
    #Sobre SOPORTE_GLOBAL_REPORTING se sobreescriben las pestañas con los valores que están linkados a las pestañas con los gráficos

    book = load_workbook('SOPORTE_GLOBAL_REPORTING.xlsx')
    writer = pd.ExcelWriter('SOPORTE_GLOBAL_REPORTING.xlsx', engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    
    # Añadir una pestaña auxiliar para que al borrarlas al menos en el Excel quede una y no salte un error
    pestana_blanco = pd.DataFrame(data={})
    pestana_blanco.to_excel(writer, 'aux_sheet')
    writer.save()

    #Borrar pestañas con datos
    book.remove_sheet(book.get_sheet_by_name('Soporte_Resumen'))
    book.remove_sheet(book.get_sheet_by_name('Soporte_Distri_Bancos'))
    book.remove_sheet(book.get_sheet_by_name('Soporte_Distri_ActyDiv'))
    book.remove_sheet(book.get_sheet_by_name('Soporte_Distri_País'))
    book.remove_sheet(book.get_sheet_by_name('Soporte_Distri_Sec'))
    book.remove_sheet(book.get_sheet_by_name('Soporte_Liquidez'))  
    book.remove_sheet(book.get_sheet_by_name('Soporte_Disti_RF'))   
    book.remove_sheet(book.get_sheet_by_name('Soporte_df_bonos'))
    book.remove_sheet(book.get_sheet_by_name('Soporte_df_preferentes'))
    book.remove_sheet(book.get_sheet_by_name('Soporte_fi_rf'))
    book.remove_sheet(book.get_sheet_by_name('Soporte_etf_rf'))
    book.remove_sheet(book.get_sheet_by_name('Soporte_Disti_RV'))
    book.remove_sheet(book.get_sheet_by_name('Soporte_df_acciones'))
    book.remove_sheet(book.get_sheet_by_name('Soporte_fi_rv'))
    book.remove_sheet(book.get_sheet_by_name('Soporte_etf_rv'))
    book.remove_sheet(book.get_sheet_by_name('Soporte_mat_div'))
    book.remove_sheet(book.get_sheet_by_name('Soporte_mat_cup'))
    book.remove_sheet(book.get_sheet_by_name('Soporte_mat_cup_forward'))
    book.remove_sheet(book.get_sheet_by_name('Extracción Datos'))
    writer.save()

    #Crearlas nuevamente para que estén vacías
    book.create_sheet('Soporte_Resumen')
    book.create_sheet('Soporte_Distri_Bancos')
    book.create_sheet('Soporte_Distri_ActyDiv')
    book.create_sheet('Soporte_Distri_País')
    book.create_sheet('Soporte_Distri_Sec')
    book.create_sheet('Soporte_Liquidez')
    book.create_sheet('Soporte_Disti_RF')
    book.create_sheet('Soporte_df_bonos')
    book.create_sheet('Soporte_df_preferentes')
    book.create_sheet('Soporte_fi_rf')
    book.create_sheet('Soporte_etf_rf')
    book.create_sheet('Soporte_Disti_RV')
    book.create_sheet('Soporte_df_acciones')
    book.create_sheet('Soporte_fi_rv')
    book.create_sheet('Soporte_etf_rv')
    book.create_sheet('Soporte_mat_div')
    book.create_sheet('Soporte_mat_cup')
    book.create_sheet('Soporte_mat_cup_forward')
    book.create_sheet('Extracción Datos')
    writer.save()

    #Borrar la pestaña auxuliar
    book.remove_sheet(book.get_sheet_by_name('aux_sheet'))
    writer.save()    
    
    #Vuelve a cargar el Excel
    book = load_workbook('SOPORTE_GLOBAL_REPORTING.xlsx')
    writer = pd.ExcelWriter('SOPORTE_GLOBAL_REPORTING.xlsx', engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    #Cargar valores nuevos en las pestañas del Excel SOPORTE_GLOBAL_REPORTING
    global_mat.to_excel(writer, 'Extracción Datos')
    
    df_liquidez.to_excel(writer, sheet_name='Soporte_Liquidez', startcol=0, index=False)
    
    df_rf_dist_activ.to_excel(writer, sheet_name='Soporte_Disti_RF', startcol=0, index=False)
    df_rf_dist_pais.to_excel(writer, sheet_name='Soporte_Disti_RF', startcol=5, index=False)
    df_rf_dist_sector.to_excel(writer, sheet_name='Soporte_Disti_RF', startcol=9, index=False)    

    df_bonos.to_excel(writer, 'Soporte_df_bonos', index=False)
    df_preferentes.to_excel(writer, 'Soporte_df_preferentes', index=False)
    df_fi_rf.to_excel(writer, 'Soporte_fi_rf', index=False)
    df_etf_rf.to_excel(writer, 'Soporte_etf_rf', index=False)

    df_rv_dist_activ.to_excel(writer, sheet_name='Soporte_Disti_RV', startcol=0, index=False)
    df_rv_dist_pais.to_excel(writer, sheet_name='Soporte_Disti_RV', startcol=5, index=False)
    df_rv_dist_sector.to_excel(writer, sheet_name='Soporte_Disti_RV', startcol=9, index=False)    
    df_acciones.to_excel(writer, 'Soporte_df_acciones', index=False)
    df_fi_rv.to_excel(writer, 'Soporte_fi_rv', index=False)
    df_etf_rv.to_excel(writer, 'Soporte_etf_rv', index=False)

    try:
        #Adaptación de la matriz al formato del GR
        def_mat_div = pd.merge(def_mat_div, global_mat[['security', 'Título']].rename(columns = {'security': 'Ticker'}).drop_duplicates(keep='first'), how='left', on=['Ticker']).drop(['Ticker'], axis=1)
        def_mat_div = def_mat_div[['Título'] + [str(i) for i in range(1,13)] + ['Total']] #Ordenar la matriz
        def_mat_div = def_mat_div.rename(columns = {'1': 'Ene', '2': 'Feb', '3': 'Mar', '4': 'Abr', '5': 'May', '6': 'Jun', '7': 'Jul', '8': 'Ago', '9': 'Sept', '10': 'Oct', '11': 'Nov', '12': 'Dic'})
        def_mat_div['Título'] = def_mat_div['Título'].fillna('Total')        
        def_mat_div.to_excel(writer, 'Soporte_mat_div', index=False)
    except:
        pestana_blanco.to_excel(writer, 'Soporte_mat_div')

    try:
        #Adaptación de la matriz al formato del GR
        def_mat_cup = pd.merge(def_mat_cup, global_mat[['security', 'Título', 'Cupón', 'Vencimiento']].rename(columns = {'security': 'Ticker'}).drop_duplicates(keep='first'), how='left', on=['Ticker']).drop(['Ticker'], axis=1)
        def_mat_cup = def_mat_cup[['Título', 'Cupón', 'Vencimiento'] + [str(i) for i in range(1,13)] + ['Total']] #Ordenar la matriz
        def_mat_cup = def_mat_cup.rename(columns = {'1': 'Ene', '2': 'Feb', '3': 'Mar', '4': 'Abr', '5': 'May', '6': 'Jun', '7': 'Jul', '8': 'Ago', '9': 'Sept', '10': 'Oct', '11': 'Nov', '12': 'Dic'})
        def_mat_cup['Título'] = def_mat_cup['Título'].fillna('Total')        
        def_mat_cup.to_excel(writer, 'Soporte_mat_cup', index=False)
    except:
        pestana_blanco.to_excel(writer, 'Soporte_mat_cup')

    try:
        #Adaptación de la matriz al formato del GR
        def_forward_mat_cup = pd.merge(def_forward_mat_cup, global_mat[['security', 'Título', 'Cupón', 'Vencimiento']].rename(columns = {'security': 'Ticker'}).drop_duplicates(keep='first'),
                                       how='left',
                                       on=['Ticker']).drop(['Ticker'], axis=1)
        def_forward_mat_cup = def_forward_mat_cup[['Título', 'Cupón', 'Vencimiento'] + [str(i) for i in range(1,13)] + ['Total']] #Ordenar la matriz
        def_forward_mat_cup = def_forward_mat_cup.rename(columns = {'1': 'Ene', '2': 'Feb', '3': 'Mar', '4': 'Abr', '5': 'May', '6': 'Jun', '7': 'Jul', '8': 'Ago', '9': 'Sept', '10': 'Oct', '11': 'Nov', '12': 'Dic'})
        def_forward_mat_cup['Título'] = def_forward_mat_cup['Título'].fillna('Total')        
        def_forward_mat_cup.to_excel(writer, 'Soporte_mat_cup_forward', index=False)
    except:
        pestana_blanco.to_excel(writer, 'Soporte_mat_cup_forward')

    dist_global_pais.to_excel(writer, 'Soporte_Distri_País', index=False)
    dist_global_sector.to_excel(writer, 'Soporte_Distri_Sec', index=False)

    df_agg_asset_type[['DISTRIBUCIÓN POR ACTIVO FINANCIERO', 'Fecha Inicial', 'Variación', 'Fecha Final', '% del total']].to_excel(writer, sheet_name='Soporte_Distri_ActyDiv', startcol=0, index=False)
    df_agg_asset_type.loc[df_agg_asset_type['DISTRIBUCIÓN POR ACTIVO FINANCIERO'].isin(['1 - Liquidez',
                                                                                        '2 - Renta Fija',
                                                                                        '3 - Renta Variable'])][['DISTRIBUCIÓN POR ACTIVO FINANCIERO','Fecha Final']].to_excel(writer, sheet_name='Soporte_Distri_ActyDiv', startcol=9, index=False)
    df_divisas[['DISTRIBUCÓN POR DIVISA', 'Fecha Inicial', 'Variación', 'Fecha Final', '% del total']].to_excel(writer, sheet_name='Soporte_Distri_ActyDiv', startrow=13, index=False)

    def_agg_bancos.to_excel(writer, sheet_name='Soporte_Distri_Bancos', startcol=0, index=False)
    df_bancos[['Banco', 'Fecha Final']].to_excel(writer, sheet_name='Soporte_Distri_Bancos', startcol=9, index=False)

    def_mat_resumen.to_excel(writer, 'Soporte_Resumen', index=False)
        
    writer.save()      
    
    print_message_box.value = 'GLOBAL REPORTING GENERADO EN: ' + str(os.getcwd()) + '\SOPORTE_GLOBAL_REPORTING.xlsx ' 

    
def One_Pager_Generation (tgt = None):
    global agg_global_data_mat
    global tot_div
    global tot_cup

    for i_bank in agg_global_data_mat['Banco'].unique():
        aux_agg_global_data_mat = agg_global_data_mat.loc[agg_global_data_mat['Banco'].isin([i_bank])]

        #Se declaran los dfs necesarios para generar el One Pager
        df_fi = pd.DataFrame(data={})
        df_eq = pd.DataFrame(data={})
        df_cash = pd.DataFrame(data={})
        df_treasuries = pd.DataFrame(data={})
        df_cash_otros = pd.DataFrame(data={})
        df_otros = pd.DataFrame(data={})

        aux_agg_global_data_mat = agg_global_data_mat.loc[agg_global_data_mat['Banco'].isin([i_bank])]
        aux_agg_global_data_mat['Variación Precio'] = aux_agg_global_data_mat['Precio Actual'] / aux_agg_global_data_mat['Precio compra'] - 1 

        if len(aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['Bonos', 'Fondos RF', 'ETFs RF','Preferentes'])]) > 0:
            
            df_fi = aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['Bonos', 'Fondos RF','ETFs RF','Preferentes'])]
            
            # Drop FI Short Dur --> Cash
            df_fi = df_fi[df_fi.Maturity_Band_Focus != 'Ultra Short'][['Clasificación', 'Rating', 'País', 'security','Título','Divisa', 'Cupón', 'Vencimiento', 'Retorno Obtenido',
                                                                       '% Total Cartera', 'Variación Precio', 'Nominal','Fecha Compra', 'Precio compra', 'Precio Actual',
                                                                       'Dividendos+Intereses', 'Valor Actual','Valor de Compra']]
            
            df_fi = df_fi.rename(columns = {'Dividendos+Intereses': 'Intereses recibidos'})
            
            #Calcular Rating medio
            aux_rating = aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['Bonos','Preferentes']), ['Rating', 'Valor Actual']]
            
            list_rtg = aux_rating['Rating'].tolist()
            aux_list_rtg = []
            for i_rtg in list_rtg:
                aix_i = i_rtg.split(' *')
                aux_list_rtg += [aix_i[0]]
            list_rtg = aux_list_rtg    
            aux_list_rtg = []
            for i_rtg in list_rtg:
                aix_i = i_rtg.split('u')
                aux_list_rtg += [aix_i[0]]

            aux_rating['Clean_RTG'] = aux_list_rtg            
            
            #Generar un diccionario con el rating y su valor correspondiente y otro diccionario que sea el inverso de este
            rating_valor = {'AAA': 21, 'AA+': 20, 'AA': 19, 'AA-': 18, 'A+': 17, 'A': 16, 'A-': 15,
                            'BBB+': 14, 'BBB': 13, 'BBB-': 12, 'BB+': 11, 'BB': 10, 'BB-': 9, 'B+': 8, 'B': 7, 'B-': 6,
                            'CCC+': 5, 'CCC': 4, 'CCC-': 3, 'CC': 2, 'C': 1, 'D': 0}
            valor_rating = {'21': 'AAA', '20': 'AA+', '19': 'AA', '18': 'AA-', '17': 'A+', '16': 'A', '15': 'A-',
                            '14': 'BBB+', '13': 'BBB', '12': 'BBB-', '11': 'BB+', '10': 'BB', '9': 'BB-', '8': 'B+', '7': 'B', '6': 'B-',
                            '5': 'CCC+', '4': 'CCC', '3': 'CCC-', '2': 'CC', '1': 'C', '0': 'D'}    

            #Asignar el número de rating a cada bono
            aux_rating['Numero_Rating'] = aux_rating['Clean_RTG'].map(rating_valor).fillna(0)     
            rating_medio = valor_rating[str(int(np.round(sum((aux_rating['Valor Actual']/aux_rating['Valor Actual'].sum()) * aux_rating['Numero_Rating']), 0)))]    

            df_fi = df_fi.sort_values(by=['% Total Cartera'], ascending=False)

            #Añadir filas en blanco para cuadrar la longitud de la matriz con el formato del Excel        
            aux_len = 47 - len(df_fi)

            if aux_len>0:
                null_list = [np.nan]*(aux_len)      

                df_fi = df_fi.append(pd.DataFrame(data = {'Clasificación': null_list, 'Rating': null_list, 'País': null_list, 'security': null_list, 'Título': null_list, 'Divisa': null_list, 'Cupón': null_list, 'Vencimiento': null_list,
                                                          'Retorno Obtenido': null_list, '% Total Cartera': null_list, 'Variación Precio': null_list, 'Nominal': null_list, 'Fecha Compra': null_list, 'Precio compra': null_list,
                                                          'Precio Actual': null_list, 'Intereses recibidos': null_list, 'Valor Actual': null_list}), ignore_index=True)        

            #Añadir la última fila con totales        
            df_fi = df_fi.append(pd.DataFrame(data = {'Clasificación': 'SUMA / MEDIA ', 'Rating': rating_medio,
                                                      'País': '', 'security': '', 'Título': '', 'Divisa': '',
                                                      'Cupón': sum((df_fi[df_fi['Clasificación'].isin(['Bonos','Preferentes'])]['Valor Actual'].fillna(0) / df_fi[df_fi['Clasificación'].isin(['Bonos','Preferentes'])]['Valor Actual'].fillna(0).sum())
                                                                   *df_fi[df_fi['Clasificación'].isin(['Bonos','Preferentes'])]['Cupón'].fillna(0)),
                                                      'Vencimiento': '',
                                                      'Retorno Obtenido': sum((df_fi['Valor de Compra'].fillna(0)/df_fi['Valor de Compra'].fillna(0).sum()) * df_fi['Retorno Obtenido'].fillna(0)),
                                                      '% Total Cartera': df_fi['% Total Cartera'].sum(),
                                                      'Variación Precio': sum((df_fi['Valor Actual'].fillna(0)/df_fi['Valor Actual'].fillna(0).sum()) * df_fi['Variación Precio'].fillna(0)),
                                                      'Nominal': '', 'Fecha Compra': '', 'Precio compra': '','Precio Actual': '',
                                                      'Intereses recibidos': df_fi['Intereses recibidos'].fillna(0).sum(),
                                                      'Valor Actual': df_fi['Valor Actual'].fillna(0).sum()}, index = [0]), ignore_index=True)

            df_fi = df_fi[['Clasificación', 'Rating', 'País', 'security', 'Título', 'Divisa', 'Cupón', 'Vencimiento', 'Retorno Obtenido', '% Total Cartera', 'Variación Precio', 'Nominal',
                           'Fecha Compra', 'Precio compra', 'Precio Actual', 'Intereses recibidos', 'Valor Actual']]    

        if len(aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['Acciones', 'Fondos RV', 'ETFs RV'])]) > 0:
            df_eq = aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['Acciones', 'Fondos RV', 'ETFs RV'])][['Clasificación', 'security', 'Título', 'Divisa',
                                                                                                                                      'Retorno Obtenido', '% Total Cartera',
                                                                                                                                      'Variación Precio', 'Nominal', 'Fecha Compra',
                                                                                                                                      'Precio compra', 'Precio Actual',
                                                                                                                                      'Dividendos+Intereses', 'Valor Actual', 'Valor de Compra',
                                                                                                                                      'Dividend yield fecha compra', 'Precio compra Report']]
            
            df_eq = df_eq.rename(columns = {'Dividendos+Intereses': 'Dividendos recibidos'})

#             df_eq['Dividend yield fecha compra'] = (df_eq['Dividendos recibidos'] / df_eq['Nominal']) / (df_eq['Precio compra Report']) 
            df_eq = df_eq.drop(['Precio compra Report'], axis=1)
            df_eq = df_eq.sort_values(by=['% Total Cartera'], ascending=False)

            #Añadir filas en blanco para cuadrar la lonitud de la matriz con el formato del Excel        
            aux_len = 47 - len(df_eq)

            if aux_len>0:
                null_list = [np.nan]*(aux_len)                   
                df_eq = df_eq.append(pd.DataFrame(data = {'Clasificación': null_list, 'security': null_list, 'Título': null_list, 'Divisa': null_list,
                                                          'Retorno Obtenido': null_list, '% Total Cartera': null_list, 'Variación Precio': null_list,
                                                          'Nominal': null_list, 'Fecha Compra': null_list, 'Precio compra': null_list, 'Precio Actual': null_list,
                                                          'Dividendos recibidos': null_list, 'Valor Actual': null_list, 'Dividend yield fecha compra': null_list}), ignore_index=True)        

            #Añadir la última fila con totales
            df_eq = df_eq.append(pd.DataFrame(data = {'Clasificación': 'SUMA / MEDIA ', 'security': '', 'Título': '', 'Divisa': '',
                                                      'Retorno Obtenido': sum((df_eq['Valor de Compra'].fillna(0)/df_eq['Valor de Compra'].fillna(0).sum()) * df_eq['Retorno Obtenido'].fillna(0)),
                                                      '% Total Cartera': df_eq['% Total Cartera'].fillna(0).sum(),
                                                      'Variación Precio': sum((df_eq['Valor Actual'].fillna(0)/df_eq['Valor Actual'].fillna(0).sum()) * df_eq['Variación Precio'].fillna(0)),
                                                      'Nominal': '', 'Fecha Compra': '', 'Precio compra': '','Precio Actual': '',
                                                      'Dividendos recibidos': df_eq['Dividendos recibidos'].fillna(0).sum(), 'Valor Actual': df_eq['Valor Actual'].fillna(0).sum(),
                                                      'Dividend yield fecha compra': sum((df_eq['Valor Actual'].fillna(0)/df_eq['Valor de Compra'].fillna(0).sum()) * df_eq['Dividend yield fecha compra'].fillna(0))}, index = [0]), ignore_index=True)

            df_eq = df_eq[['Clasificación', 'security', 'Título', 'Divisa', 'Retorno Obtenido', '% Total Cartera', 'Variación Precio', 'Nominal', 'Fecha Compra', 'Precio compra', 'Precio Actual', 'Dividendos recibidos',
                           'Valor Actual', 'Dividend yield fecha compra']]

        if len(aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['CASH'])]) > 0:
            # CASH - Currencies 
            df_cash = aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['CASH'])]
            df_cash = df_cash[['Clasificación', 'Título', 'Divisa','Retorno Obtenido', '% Total Cartera', 'Valor Actual','Valor de Compra']]
            df_cash = df_cash.sort_values(by=['% Total Cartera'], ascending=False)

            #Añadir filas en blanco para cuadrar la lonitud de la matriz con el formato del Excel    
            aux_len = 7 - len(df_cash)

            if aux_len>0:
                null_list = [np.nan]*(aux_len)        
                df_cash = df_cash.append(pd.DataFrame(data = {'Clasificación': null_list, 'Título': null_list, 'Divisa': null_list, 'Retorno Obtenido': null_list, '% Total Cartera': null_list, 'Valor Actual': null_list}), ignore_index=True)

            #Añadir la última fila con totales                
            df_cash = df_cash.append(pd.DataFrame(data = {'Clasificación': 'SUMA / MEDIA ', 'Título': '', 'Divisa': '', 
                                                          'Retorno Obtenido': sum((df_cash['Valor de Compra'].fillna(0)/df_cash['Valor de Compra'].fillna(0).sum()) * df_cash['Retorno Obtenido'].fillna(0)),
                                                          '% Total Cartera': df_cash['% Total Cartera'].fillna(0).sum(), 
                                                          'Valor Actual': df_cash['Valor Actual'].fillna(0).sum()}, index = [0]), ignore_index=True)    

            df_cash = df_cash[['Clasificación', 'Título', 'Divisa', 'Retorno Obtenido', '% Total Cartera', 'Valor Actual']]
            
            
        if len(aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['Tesoros'])]) > 0: 
            # CASH - Treasuries
            df_treasuries = aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['Tesoros'])]
            df_treasuries = df_treasuries[['Clasificación', 'Rating', 'País', 'Título','Divisa', 'Cupón', 'Vencimiento', 'Retorno Obtenido','% Total Cartera', 'Variación Precio', 'Nominal','Fecha Compra',
                                           'Precio compra', 'Precio Actual','Dividendos+Intereses', 'Valor Actual','Valor de Compra']]
            df_treasuries = df_treasuries.rename(columns = {'Dividendos+Intereses': 'Intereses recibidos'})

            #Calcular Rating medio
            try: 
                aux_rating = aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['Tesoros']), ['Rating', 'Valor Actual']]
                
                list_rtg = aux_rating['Rating'].tolist()
                aux_list_rtg = []
                for i_rtg in list_rtg:
                    aix_i = i_rtg.split(' *')
                    aux_list_rtg += [aix_i[0]]
                list_rtg = aux_list_rtg    
                aux_list_rtg = []
                for i_rtg in list_rtg:
                    aix_i = i_rtg.split('u')
                    aux_list_rtg += [aix_i[0]]

                aux_rating['Clean_RTG'] = aux_list_rtg                   
                
                #Generar un diccionario con el rating y su valor correspondiente y otro diccionario que sea el inverso de este
                rating_valor = {'AAA': 21, 'AA+': 20, 'AA': 19, 'AA-': 18, 'A+': 17, 'A': 16, 'A-': 15,
                                'BBB+': 14, 'BBB': 13, 'BBB-': 12, 'BB+': 11, 'BB': 10, 'BB-': 9, 'B+': 8, 'B': 7, 'B-': 6,
                                'CCC+': 5, 'CCC': 4, 'CCC-': 3, 'CC': 2, 'C': 1, 'D': 0}
                valor_rating = {'21': 'AAA', '20': 'AA+', '19': 'AA', '18': 'AA-', '17': 'A+', '16': 'A', '15': 'A-',
                                '14': 'BBB+', '13': 'BBB', '12': 'BBB-', '11': 'BB+', '10': 'BB', '9': 'BB-', '8': 'B+', '7': 'B', '6': 'B-',
                                '5': 'CCC+', '4': 'CCC', '3': 'CCC-', '2': 'CC', '1': 'C', '0': 'D'}    

                #Asignar el número de rating a cada bono
                aux_rating['Numero_Rating'] = aux_rating['Clean_RTG'].map(rating_valor).fillna(0)     
                rating_medio = valor_rating[str(int(np.round(sum((aux_rating['Valor Actual']/aux_rating['Valor Actual'].sum()) * aux_rating['Numero_Rating']), 0)))]

            except:
                rating_medio = np.nan

            df_treasuries = df_treasuries.sort_values(by=['% Total Cartera'], ascending=False)

            #Añadir filas en blanco para cuadrar la lonitud de la matriz con el formato del Excel
            aux_len = 7 - len(df_treasuries)

            if aux_len>0:
                null_list = [np.nan]*(aux_len)        
                df_treasuries = df_treasuries.append(pd.DataFrame(data = {'Clasificación': null_list, 'Rating': null_list, 'País': null_list, 'Título': null_list, 'Divisa': null_list, 'Cupón': null_list, 'Vencimiento': null_list,
                                                              'Retorno Obtenido': null_list, '% Total Cartera': null_list, 'Variación Precio': null_list, 'Nominal': null_list, 'Fecha Compra': null_list, 'Precio compra': null_list,
                                                              'Precio Actual': null_list, 'Intereses recibidos': null_list, 'Valor Actual': null_list}), ignore_index=True)        

            #Añadir la última fila con totales                
            df_treasuries = df_treasuries.append(pd.DataFrame(data = {'Clasificación': 'SUMA / MEDIA ', 'Rating': rating_medio, 'País': '', 'Título': '', 'Divisa': '',
                                                                      'Cupón': sum((df_treasuries['Valor Actual'].fillna(0)/df_treasuries['Valor Actual'].fillna(0).sum()) * df_treasuries['Cupón']), 'Vencimiento': '',
                                                                      'Retorno Obtenido': sum((df_treasuries['Valor de Compra'].fillna(0)/df_treasuries['Valor de Compra'].fillna(0).sum()) * df_treasuries['Retorno Obtenido'].fillna(0)),
                                                                      '% Total Cartera': df_treasuries['% Total Cartera'].fillna(0).sum(),
                                                                      'Variación Precio': sum((df_treasuries['Valor Actual'].fillna(0)/df_treasuries['Valor Actual'].fillna(0).sum()) * df_treasuries['Variación Precio'].fillna(0)),
                                                                      'Nominal': '', 'Fecha Compra': '', 'Precio compra': '','Precio Actual': '',
                                                                      'Intereses recibidos': df_treasuries['Intereses recibidos'].fillna(0).sum(),
                                                                      'Valor Actual': df_treasuries['Valor Actual'].fillna(0).sum()}, index = [0]), ignore_index=True)        

            df_treasuries = df_treasuries[['Clasificación', 'Rating', 'País', 'Título', 'Divisa', 'Cupón', 'Vencimiento', 'Retorno Obtenido', '% Total Cartera', 'Variación Precio', 'Nominal', 'Fecha Compra',
                                           'Precio compra', 'Precio Actual', 'Intereses recibidos', 'Valor Actual']]    
            
            
            
            

        if len(aux_agg_global_data_mat[aux_agg_global_data_mat.Maturity_Band_Focus == 'Ultra Short']) > 0:
             # CASH - Otros
            df_cash_otros = aux_agg_global_data_mat[aux_agg_global_data_mat.Maturity_Band_Focus == 'Ultra Short']
            df_cash_otros = df_cash_otros[['Clasificación', 'Rating', 'País', 'Título','Divisa', 'Cupón', 'Vencimiento', 'Retorno Obtenido','% Total Cartera', 'Variación Precio', 'Nominal','Fecha Compra',
                                           'Precio compra', 'Precio Actual','Dividendos+Intereses', 'Valor Actual','Valor de Compra']]
            df_cash_otros = df_cash_otros.rename(columns = {'Dividendos+Intereses': 'Intereses recibidos'})
            #Calcular Rating medio
            try: 
                aux_rating = aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Maturity_Band_Focus'].isin(['Ultra Short']), ['Rating', 'Valor Actual']]
                
                list_rtg = aux_rating['Rating'].tolist()
                aux_list_rtg = []
                for i_rtg in list_rtg:
                    aix_i = i_rtg.split(' *')
                    aux_list_rtg += [aix_i[0]]
                list_rtg = aux_list_rtg    
                aux_list_rtg = []
                for i_rtg in list_rtg:
                    aix_i = i_rtg.split('u')
                    aux_list_rtg += [aix_i[0]]

                aux_rating['Clean_RTG'] = aux_list_rtg                   
                
                #Generar un diccionario con el rating y su valor correspondiente y otro diccionario que sea el inverso de este
                rating_valor = {'AAA': 21, 'AA+': 20, 'AA': 19, 'AA-': 18, 'A+': 17, 'A': 16, 'A-': 15,
                                'BBB+': 14, 'BBB': 13, 'BBB-': 12, 'BB+': 11, 'BB': 10, 'BB-': 9, 'B+': 8, 'B': 7, 'B-': 6,
                                'CCC+': 5, 'CCC': 4, 'CCC-': 3, 'CC': 2, 'C': 1, 'D': 0}
                valor_rating = {'21': 'AAA', '20': 'AA+', '19': 'AA', '18': 'AA-', '17': 'A+', '16': 'A', '15': 'A-',
                                '14': 'BBB+', '13': 'BBB', '12': 'BBB-', '11': 'BB+', '10': 'BB', '9': 'BB-', '8': 'B+', '7': 'B', '6': 'B-',
                                '5': 'CCC+', '4': 'CCC', '3': 'CCC-', '2': 'CC', '1': 'C', '0': 'D'}    

                #Asignar el número de rating a cada bono
                aux_rating['Numero_Rating'] = aux_rating['Clean_RTG'].map(rating_valor).fillna(0)     
                rating_medio = valor_rating[str(int(np.round(sum((aux_rating['Valor Actual']/aux_rating['Valor Actual'].sum()) * aux_rating['Numero_Rating']), 0)))]

            except:
                rating_medio = np.nan

            df_cash_otros = df_cash_otros.sort_values(by=['% Total Cartera'], ascending=False)

            #Añadir filas en blanco para cuadrar la lonitud de la matriz con el formato del Excel
            aux_len = 7 - len(df_cash_otros)

            if aux_len>0:
                null_list = [np.nan]*(aux_len)        
                df_cash_otros = df_cash_otros.append(pd.DataFrame(data = {'Clasificación': null_list, 'Rating': null_list, 'País': null_list, 'Título': null_list, 'Divisa': null_list, 'Cupón': null_list, 'Vencimiento': null_list,
                                                              'Retorno Obtenido': null_list, '% Total Cartera': null_list, 'Variación Precio': null_list, 'Nominal': null_list, 'Fecha Compra': null_list, 'Precio compra': null_list,
                                                              'Precio Actual': null_list, 'Intereses recibidos': null_list, 'Valor Actual': null_list}), ignore_index=True)        

            #Añadir la última fila con totales                
            df_cash_otros = df_cash_otros.append(pd.DataFrame(data = {'Clasificación': 'SUMA / MEDIA ', 'Rating': rating_medio, 'País': '', 'Título': '', 'Divisa': '',
                                                                      'Cupón': sum((df_cash_otros['Valor Actual'].fillna(0)/df_cash_otros['Valor Actual'].fillna(0).sum()) * df_cash_otros['Cupón']), 'Vencimiento': '',
                                                                      'Retorno Obtenido': sum((df_cash_otros['Valor de Compra'].fillna(0)/df_cash_otros['Valor de Compra'].fillna(0).sum()) * df_cash_otros['Retorno Obtenido'].fillna(0)),
                                                                      '% Total Cartera': df_cash_otros['% Total Cartera'].fillna(0).sum(),
                                                                      'Variación Precio': sum((df_cash_otros['Valor Actual'].fillna(0)/df_cash_otros['Valor Actual'].fillna(0).sum()) * df_cash_otros['Variación Precio'].fillna(0)),
                                                                      'Nominal': '', 'Fecha Compra': '', 'Precio compra': '','Precio Actual': '',
                                                                      'Intereses recibidos': df_cash_otros['Intereses recibidos'].fillna(0).sum(),
                                                                      'Valor Actual': df_cash_otros['Valor Actual'].fillna(0).sum()}, index = [0]), ignore_index=True)        

            df_cash_otros = df_cash_otros[['Clasificación', 'Rating', 'País', 'Título', 'Divisa', 'Cupón', 'Vencimiento', 'Retorno Obtenido', '% Total Cartera', 'Variación Precio', 'Nominal', 'Fecha Compra',
                                           'Precio compra', 'Precio Actual', 'Intereses recibidos', 'Valor Actual']]                
            
            
            
            
        if len(aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['Otros'])]) > 0:    
            df_otros = aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['Otros'])][['Clasificación', 'Rating', 'País', 'Título',
                                                                                                              'Divisa', 'Cupón', 'Vencimiento', 'Retorno Obtenido',
                                                                                                              '% Total Cartera', 'Variación Precio', 'Nominal',
                                                                                                              'Fecha Compra', 'Precio compra', 'Precio Actual',
                                                                                                              'Dividendos+Intereses', 'Valor Actual','Valor de Compra']]
            
            df_otros = df_otros.rename(columns = {'Dividendos+Intereses': 'Intereses recibidos'})

            #Calcular Rating medio
            try: 
                aux_rating = aux_agg_global_data_mat.loc[aux_agg_global_data_mat['Clasificación'].isin(['Otros']), ['Rating', 'Valor Actual']]
                
                list_rtg = aux_rating['Rating'].tolist()
                aux_list_rtg = []
                for i_rtg in list_rtg:
                    aix_i = i_rtg.split(' *')
                    aux_list_rtg += [aix_i[0]]
                list_rtg = aux_list_rtg    
                aux_list_rtg = []
                for i_rtg in list_rtg:
                    aix_i = i_rtg.split('u')
                    aux_list_rtg += [aix_i[0]]

                aux_rating['Clean_RTG'] = aux_list_rtg                   
                
                #Generar un diccionario con el rating y su valor correspondiente y otro diccionario que sea el inverso de este
                rating_valor = {'AAA': 21, 'AA+': 20, 'AA': 19, 'AA-': 18, 'A+': 17, 'A': 16, 'A-': 15,
                                'BBB+': 14, 'BBB': 13, 'BBB-': 12, 'BB+': 11, 'BB': 10, 'BB-': 9, 'B+': 8, 'B': 7, 'B-': 6,
                                'CCC+': 5, 'CCC': 4, 'CCC-': 3, 'CC': 2, 'C': 1, 'D': 0}
                valor_rating = {'21': 'AAA', '20': 'AA+', '19': 'AA', '18': 'AA-', '17': 'A+', '16': 'A', '15': 'A-',
                                '14': 'BBB+', '13': 'BBB', '12': 'BBB-', '11': 'BB+', '10': 'BB', '9': 'BB-', '8': 'B+', '7': 'B', '6': 'B-',
                                '5': 'CCC+', '4': 'CCC', '3': 'CCC-', '2': 'CC', '1': 'C', '0': 'D'}    

                #Asignar el número de rating a cada bono
                aux_rating['Numero_Rating'] = aux_rating['Clean_RTG'].map(rating_valor).fillna(0)     
                rating_medio = valor_rating[str(int(np.round(sum((aux_rating['Valor Actual']/aux_rating['Valor Actual'].sum()) * aux_rating['Numero_Rating']), 0)))]

            except:
                rating_medio = np.nan

            df_otros = df_otros.sort_values(by=['% Total Cartera'], ascending=False)

            #Añadir filas en blanco para cuadrar la lonitud de la matriz con el formato del Excel
            aux_len = 7 - len(df_otros)

            if aux_len>0:
                null_list = [np.nan]*(aux_len)        
                df_otros = df_otros.append(pd.DataFrame(data = {'Clasificación': null_list, 'Rating': null_list, 'País': null_list, 'Título': null_list, 'Divisa': null_list, 'Cupón': null_list, 'Vencimiento': null_list,
                                                              'Retorno Obtenido': null_list, '% Total Cartera': null_list, 'Variación Precio': null_list, 'Nominal': null_list, 'Fecha Compra': null_list, 'Precio compra': null_list,
                                                              'Precio Actual': null_list, 'Intereses recibidos': null_list, 'Valor Actual': null_list}), ignore_index=True)        

            #Añadir la última fila con totales                
            df_otros = df_otros.append(pd.DataFrame(data = {'Clasificación': 'SUMA / MEDIA ', 'Rating': rating_medio, 'País': '', 'Título': '', 'Divisa': '',
                                                            'Cupón': sum((df_otros['Valor Actual'].fillna(0)/df_otros['Valor Actual'].fillna(0).sum()) * df_otros['Cupón']),'Vencimiento': '',
                                                            'Retorno Obtenido': sum((df_otros['Valor de Compra'].fillna(0)/df_otros['Valor de Compra'].fillna(0).sum()) * df_otros['Retorno Obtenido'].fillna(0)),
                                                            '% Total Cartera': df_otros['% Total Cartera'].fillna(0).sum(),
                                                            'Variación Precio': sum((df_otros['Valor Actual'].fillna(0)/df_otros['Valor Actual'].fillna(0).sum()) * df_otros['Variación Precio'].fillna(0)),
                                                            'Nominal': '', 'Fecha Compra': '', 'Precio compra': '','Precio Actual': '',
                                                            'Intereses recibidos': df_otros['Intereses recibidos'].fillna(0).sum(), 'Valor Actual': df_otros['Valor Actual'].fillna(0).sum()}, index = [0]), ignore_index=True)        

            df_otros = df_otros[['Clasificación', 'Rating', 'País', 'Título', 'Divisa', 'Cupón', 'Vencimiento', 'Retorno Obtenido', '% Total Cartera', 'Variación Precio', 'Nominal', 'Fecha Compra', 'Precio compra', 'Precio Actual',
                                 'Intereses recibidos', 'Valor Actual']]       


        #Se genera una copia del nuevo One Pager para el banco correspondiente
        new_bank_name = shutil.copy('SOPORTE_ONE_PAGER.xlsx', 'SOP_ONE_P_' + i_bank + '.xlsx')

        #Se carga el Excel nuevo

        book = load_workbook(new_bank_name)
        writer = pd.ExcelWriter(new_bank_name, engine='openpyxl') 
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

        # Sobreescribir las pestañas soporte del Excel
        df_fi.to_excel(writer, sheet_name='Renta Fija', index=False, header=False, startrow=3)
        df_eq.to_excel(writer, sheet_name='Renta Variable', index=False, header=False, startrow=3)
        df_cash.to_excel(writer, sheet_name='Liquidez', index=False, header=False, startrow=3)
        df_treasuries.to_excel(writer, sheet_name='Liquidez', index=False, header=False, startrow=15)
        df_cash_otros.to_excel(writer, sheet_name='Liquidez', index=False, header=False, startrow=27)
#        df_otros.to_excel(writer, sheet_name='Liquidez', index=False, header=False, startrow=27)
        
        try:
            var_exist_test = tot_div - 1
        except:
            tot_div = 0

        try:
            var_exist_test = tot_cup - 1
        except:
            tot_cup = 0        
        
        aux_tot_div = pd.DataFrame({'Tot_Dividendos':[tot_div]})
        aux_tot_cup = pd.DataFrame({'Tot_Cupones':[tot_cup]})

        aux_tot_div.to_excel(writer, sheet_name='Tot_DivyCup')
        aux_tot_cup.to_excel(writer, sheet_name='Tot_DivyCup', startcol=5)

        writer.save()  
    
    print_message_box.value = 'ONE PAGER GENERADO EN: ' + str(os.getcwd()) + '\SOP_OP_XXX NOMBRE BANCO XXXX.xlsx '  

In [3]:
######## Please wait message

wait_msg = HTML("""<h2 style="color:white;">Calculando .....</h>
    <i class="fa fa-spinner fa-spin fa-2x fa-fw" style="color:white;"></i>""")
wait_msg.layout.height = '1px'
wait_msg.layout.visibility = 'hidden'
wait_msg.layout.margin = '20px 10px 10px 10px'

############################

############# MAIN GUI ##############

#Generar espacios entre distintos widgets
aux_H_label = Label(value='') #Espacio horizontal
aux_H_label.layout.width = '30px'

aux_first_H_label = Label(value='') #Primer Espacio horizontal tipo sangría
aux_first_H_label.layout.width = '200px'

aux_V_label = Label(value='') #Espacio Vertical
aux_V_label.layout.height = '30px'

#FILA 1 COLUMNA 1

header_row1 = Label(value='$CARGA$ $INICIAL$ $DE$ $CARTERAS$')
header_row1.layout.width = '300px'

#Carga de portfolios en PRTU <GO>
port_list = bqport.list_portfolios()
port_name = []

for port_id in port_list:

    port_name.append(port_id['name'])

port = SelectMultiple(options = [])    
port = SelectMultiple(options = sorted(port_name))
port.layout.height = '300px'

#Agrupación de widgets de la FILA 1
row1 = HBox([aux_first_H_label, VBox([header_row1, port]), Box([wait_msg])])

#FILA 2
header_row2 = Label(value='$PARAMETRIZACIÓN$ $DEL$ $INFORME$')
header_row2.layout.width = '320px'

header_row2_1 = Label(value='DATOS DE ENTRADA')
header_row2_1.layout.width = '300px'

label_st = Label(value='Fecha Inicio')
label_st.layout.width = '100px'
date_st = DatePicker(date_format = '%d-%m-%Y')
date_st.layout.width = '200px'


label_end = Label(value='Fecha Fin')
label_end.layout.width = '80px'
date_end = DatePicker(date_format = '%d-%m-%Y')
date_end.layout.width = '200px'

date_end.value = datetime.datetime.today().strftime('%d-%m-%Y')
date_st.value = datetime.datetime(datetime.datetime.today().year - 1, 12, 31).strftime('%d-%m-%Y')

#Buscador de monedas
label_ac_curncy = Label(value='Moneda')
label_ac_curncy.layout.width = '60px'
ac_curncy = AutoComplete(value = 'USD', data = ['AED', 'AFN', 'ALL', 'AMD', 'ANG', 'AOA', 'ARS', 'ATS', 'AUD', 'AUd', 'AWG', 'AZM', 'AZN', 'BAM', 'BBD', 'BDT', 'BEF', 'BGN',
                                         'BHD', 'BIF', 'BMD', 'BND', 'BOB','BRL', 'BRl', 'BSD','BTN', 'BWP', 'BWp', 'BYR', 'BZD', 'CAD', 'CAd', 'CDF', 'CHF', 'CHf', 'CLF',
                                         'CLP', 'CNH', 'CNY', 'COP', 'CRC', 'CSD', 'CUP', 'CVE', 'CYP', 'CZK', 'DEM', 'DJF', 'DKK', 'DOP', 'DZD', 'ECS', 'EEK', 'EGP',
                                         'ERN', 'ESP', 'ETB', 'EUA', 'EUR', 'EUr', 'FIM', 'FJD', 'FKP', 'FRF', 'GBP', 'GBp', 'GEL', 'GHC', 'GHS', 'GIP', 'GMD', 'GNF',
                                         'GRD', 'GTQ', 'GYD', 'HKD', 'HNL', 'HRK', 'HTG', 'HUF', 'IDR', 'IEP', 'IEp', 'ILS', 'ILs', 'IMP', 'INR', 'IQD', 'IRR', 'ISK',
                                         'ITL', 'JMD', 'JOD', 'JPY', 'KES', 'KGS', 'KHR', 'KMF', 'KPW', 'KRW', 'KWD', 'KWd', 'KYD', 'KZT', 'LAK', 'LBP', 'LKR', 'LRD',
                                         'LSL', 'LTL', 'LUF', 'LVL', 'LYD', 'MAD', 'MDL', 'MGA', 'MGF', 'MKD', 'MMK', 'MNT', 'MOP', 'MRO', 'MTL', 'MUR', 'MVR', 'MWK',
                                         'MWk', 'MXN', 'MYR', 'MYr', 'MZM', 'MZN', 'NAD', 'NAd', 'NGN', 'NIO', 'NLG', 'NOK', 'NPR', 'NZD', 'OMR', 'PAB', 'PEN', 'PGK',
                                         'PHP', 'PKR', 'PLN', 'PTE', 'PYG', 'QAR', 'ROL', 'RON', 'RSD', 'RUB', 'RWF', 'SAR', 'SBD', 'SCR', 'SDG', 'SEK', 'SGD', 'SGd',
                                         'SIT', 'SKK', 'SLL', 'SOS', 'SPL', 'SRG', 'SSP', 'STD', 'SVC', 'SYP', 'SZL', 'SZl', 'THB', 'THO', 'TJS', 'TMM', 'TND', 'TOP',
                                         'TRL', 'TRY', 'TTD', 'TWD', 'TZS', 'UAH', 'UDI', 'UGX', 'USD', 'USd', 'UYI', 'UYU', 'UZS', 'VEB', 'VEF', 'VND', 'VUV', 'WST',
                                         'XAF', 'XAU', 'XCD', 'XDR', 'XEU', 'XOF', 'XPF', 'YER', 'ZAR', 'ZAr', 'ZMK', 'ZMW', 'ZMk', 'ZWD', 'ZWR', 'ZWd'])
ac_curncy.layout.width = '150px'


#Botón para aplicar parámetros introducidos
request_button2 = Button(description = 'CARGAR', button_style = 'warning')
request_button2.on_click(download_main_data)
request_button2.layout.width = '150px'

#Agrupación de widgets de la FILA 1
date_st_widget = HBox([label_st, date_st])
date_end_widget = HBox([label_end, date_end])
curncy_widget = HBox([label_ac_curncy, ac_curncy])
row2 = HBox([aux_first_H_label, VBox([header_row2, aux_V_label, header_row2_1, HBox([date_st_widget, date_end_widget, curncy_widget, request_button2])])])

#FILA 3

header_row3 = Label(value='INCLUIR PATRIMONIO CARTERAS MANEJADAS')
header_row3.layout.width = '350px'

label_port_patri = Label(value='Cartera Manejada')
label_port_patri.layout.width = '120px'
port_patri = AutoComplete(data=[])
port_patri.layout.width = '200px'

label_patri_st_date = Label(value='Patrimonio FI')
label_patri_st_date.layout.width = '100px'
patri_st_date = Text(value='')
patri_st_date.layout.width = '150px'

label_patri_end_date = Label(value='Patrimonio FF')
label_patri_end_date.layout.width = '100px'
patri_end_date = Text(value='')
patri_end_date.layout.width = '150px'

#Botón para añadir carteras manejadas
request_button3 = Button(description = 'AGREGAR', button_style = 'warning')
request_button3.on_click(gen_carteras_manejadas)
request_button3.layout.width = '120px'

#Agrupación de widgets de la FILA 3
row3 = HBox([aux_first_H_label, VBox([header_row3, HBox([label_port_patri, port_patri, label_patri_st_date, patri_st_date, label_patri_end_date, patri_end_date, request_button3])])])

#FILA 4

header_row4 = Label(value='INCLUIR DEPÓSITOS Y RETIROS NETOS')
header_row4.layout.width = '350px'

label_port_patri = Label(value='Cartera')
label_port_patri.layout.width = '120px'

port_depo_ret = Dropdown(options=[])
port_depo_ret.layout.width = '200px'

depo_ret = Text(value='')

#Botón para añadir info depósitos/retiros
request_button4 = Button(description = 'AGREGAR', button_style = 'warning')
request_button4.on_click(generate_dep_ret)
request_button4.layout.width = '334px'

#Agrupación de widgets de la FILA 4
row4 = HBox([aux_first_H_label, VBox([header_row4, HBox([label_port_patri, port_depo_ret, depo_ret, request_button4])])])

#FILA 5

header_row5 = Label(value='$VALIDACION$ $DE$ $DATOS$')
header_row5.layout.width = '300px'

#Agrupación de widgets de la FILA 5
row5 = HBox([aux_first_H_label, VBox([header_row5])])

#FILA 6.0
read_excel_button = Button(description = 'LEER EXCEL DE PRECIOS', button_style = 'warning')
read_excel_button.on_click(Read_Prices_from_Excel)
read_excel_button.layout.width = '965px'

#FILA 6

label_assets_class = Label(value='Clasificación Tipo Activo')
label_assets_class.layout.width = '150px'

assets_class = Dropdown(options=[])
assets_class.observe(gen_excel_type_table, names = 'value')
assets_class.layout.width = '200px'

output_number_assets_class = Text(value='', disabled = True) #Esta ventana de texto es para mostrar el número de activos que hay de cada categoría
output_number_assets_class.layout.width = '30px'

row6 = VBox([HBox([aux_first_H_label, read_excel_button]), aux_V_label, HBox([aux_first_H_label, label_assets_class, assets_class, output_number_assets_class])])

#FILA 7

#Se va a generar el widget para incluir la tabla vacía
tabla_excel = VBox([])

#FILA 8

#Botón para aplicar las modificaciones de la tabla en la matriz de datos original
request_button5 = Button(description = 'APLICAR MODIFICACIONES', button_style = 'danger')
request_button5.on_click(apply_changes_excel_type_table)
request_button5.layout.width = '965px'

#FILA 9

#Botón para pasar a la última fase en la que se genere el Excel
request_button6 = Button(description = 'DATOS VALIDADOS', button_style = 'success')
request_button6.on_click(validate_data)
request_button6.layout.width = '965px'

#FILA 10 

#Botón para descargar datos para la generación del Global Reporting
request_button7 = Button(description = 'DESCARGAR GLOBAL REPORTING', button_style = 'info')
request_button7.on_click(Global_Reporting_Generation)
request_button7.layout.width = '480px'
request_button7.layout.height = '50px'

#Botón para descargar datos para la generación del One Pager
request_button8 = Button(description = 'DESCARGAR ONE PAGER', button_style = 'success')
request_button8.on_click(One_Pager_Generation)
request_button8.layout.width = '480px'
request_button8.layout.height = '50px'

#FILA 11

#Cuadro donde se imprimirá texto que guíe al usuario, (principalemnte para indicar la tipología de los errores)
print_message_box = Text(value='')
print_message_box.layout.width = '965px'

#####################################

#Agrupación de todos los widgets
hidden_widgets1 = VBox([row2, aux_V_label]) 

#Para guiar más al usuario, se ocultan estos widgets que se verán una vez se pulse el botón "CARGAR"
hidden_widgets2 = VBox([row3, aux_V_label, row4, aux_V_label, row5, aux_V_label, row6, aux_V_label, tabla_excel, aux_V_label, HBox([aux_first_H_label, VBox([request_button5, request_button6])]), aux_V_label])
hidden_widgets2.layout.visibility = 'hidden'

hidden_widgets3 = HBox([aux_first_H_label, request_button7, request_button8])
hidden_widgets3.layout.visibility = 'hidden' 

main = VBox([row1, aux_V_label, hidden_widgets1, hidden_widgets2, hidden_widgets3, aux_V_label, HBox([aux_first_H_label, print_message_box])])
main

In [4]:
download_main_data()
# Global_Reporting_Generation()
# def_mat_cup
# global_mat
# def_forward_mat_cup
# One_Pager_Generation()
# def_mat_div
# validate_data()
# apply_changes_excel_type_table (tgt = None)

In [2]:
import os
print(os.getcwd())

C:\Users\N360716\AppData\Local\bipy\28942540\projects\d79fa1e593dc4dcc8560a25536e0861a
